# Durchführung eines Data Mining Prozesses 

## Bibliotheken Import

In [1]:
## IMPORTS
import gitlab
import pickle
import numpy as np
import git
import json
import os
import re 
import matplotlib.pyplot as plt
from tabulate import tabulate
from operator import itemgetter
import time
import datetime
from pathlib import Path
import ndjson
# from guesslang import Guess
from colorama import Fore, Back, Style
import operator
import mysql.connector
import subprocess
import xml.etree.ElementTree as ET
import threading
import matplotlib 
from matplotlib.dates import DateFormatter
import dateutil.parser
import locale
import calendar
from mycolorpy import colorlist as mcp
import collections
from wordcloud import WordCloud
import functools
from itertools import combinations
import math
import itertools
import matplotlib.dates
font = {"fontname":"Modern Computer"}
locale.setlocale(locale.LC_ALL, 'de_DE')
from scipy.spatial import distance

## Initialisierung des Reporitories

In [2]:
### INITIAL INFO
name = "inkscape"
branch = "master"
repo_path_local = ">LOKALER REPOSITORY PFAD<" 

## Gitlab Python (API)
gl = gitlab.Gitlab('https://gitlab.com', private_token='>PRIVATER TOKEN<', api_version=4)
gl.auth()
project = gl.projects.get('inkscape/inkscape')
group = gl.groups.get(name)

## Initialisierung der Datenbank

In [3]:
#initial database commands
mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<")
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS "+name)
mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
mycursor = mydb.cursor()
#initial table commands

mycursor.execute("CREATE TABLE IF NOT EXISTS commits (id INT AUTO_INCREMENT PRIMARY KEY, hexsha VARCHAR(256), author VARCHAR(256), author_mail VARCHAR(256), authoredDate DATETIME, authoredDate_real DATETIME, committer VARCHAR(256), committer_mail VARCHAR(256), committed_date DATETIME,  committed_date_real DATETIME, message LONGTEXT)")
mycursor.execute("CREATE TABLE IF NOT EXISTS stats (stats_id INT AUTO_INCREMENT PRIMARY KEY, commit_id INT, file_name VARCHAR(256), insertions INT, deletions INT)")
mycursor.execute("CREATE TABLE IF NOT EXISTS files (id INT AUTO_INCREMENT PRIMARY KEY, file_name VARCHAR(256), ending VARCHAR(256), init_CLOC INT, categorie VARCHAR(256))")
mycursor.execute("CREATE TABLE IF NOT EXISTS couplings (id INT AUTO_INCREMENT PRIMARY KEY, file_name1 VARCHAR(256), file_name2 VARCHAR(256), count INT, categorie1 VARCHAR(256), categorie2 VARCHAR(256))")
mycursor.execute("CREATE TABLE IF NOT EXISTS issues (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(256), description LONGTEXT, state VARCHAR(256), author VARCHAR(256), created_at DATETIME, closed_at DATETIME, closed_by VARCHAR(256), labels VARCHAR(256))")
mycursor.execute("CREATE TABLE IF NOT EXISTS releases (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(256), description LONGTEXT, released_at DATETIME, author VARCHAR(256), commit_id VARCHAR(256))")
mycursor.execute("CREATE TABLE IF NOT EXISTS labels (id INT AUTO_INCREMENT PRIMARY KEY, label_id INT, name VARCHAR(256), description MEDIUMTEXT)")
mycursor.execute("CREATE TABLE IF NOT EXISTS merge_requests (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(256), description LONGTEXT, state VARCHAR(256), merged_by VARCHAR(256), merged_at DATETIME, merge_status VARCHAR(256))")
mycursor.execute("CREATE TABLE IF NOT EXISTS cppcheck (id INT AUTO_INCREMENT PRIMARY KEY, commit_id INT, file_name VARCHAR(256), line INT, categorie VARCHAR(256), fehler VARCHAR(256), msg MEDIUMTEXT)")
mycursor.execute("CREATE TABLE IF NOT EXISTS flawfinder (id INT AUTO_INCREMENT PRIMARY KEY, commit_id INT, file_name VARCHAR(256), line INT, DefaultLevel INT, Level INT, categorie VARCHAR(256), name VARCHAR(256), msg MEDIUMTEXT)")

In [4]:
'''
commits: hexsha, author, authoredDate, committer, committed_date, message, tree
stats: commit_id, file_name, insertions, deletions, total
comitter: name, in_code, del_code, in_kommi, del_kommi, in_leer, del_leer
files: file_name, ending, init_CLOC
couplings: file_name1, file_name2, count
issues: title, description, state, author, created_at, closed_at, closed_by, labels
releases: name, description, released_at, author, commit_id
labels: label_id, name, description
merge_requests: title, description, state, merged_by, merged_at, merge_status
cppcheck: commit_id, file_name, line, categorie, fehler, msg
flawfinder: commit_id, file_name, line, DefaultLevel, Level, categorie, name, msg 
'''

'\ncommits: hexsha, author, authoredDate, committer, committed_date, message, tree\nstats: commit_id, file_name, insertions, deletions, total\ncomitter: name, in_code, del_code, in_kommi, del_kommi, in_leer, del_leer\nfiles: file_name, ending, init_CLOC\ncouplings: file_name1, file_name2, count\nissues: title, description, state, author, created_at, closed_at, closed_by, labels\nreleases: name, description, released_at, author, commit_id\nlabels: label_id, name, description\nmerge_requests: title, description, state, merged_by, merged_at, merge_status\ncppcheck: commit_id, file_name, line, categorie, fehler, msg\nflawfinder: commit_id, file_name, line, DefaultLevel, Level, categorie, name, msg \n'

## Repository auf aktuellsten Stand bringen

In [5]:
## git (local Operations)
# Pull
repo = git.Repo(repo_path_local)
repo.git.checkout("master")
repo.git.pull()
head = repo.heads[0]
head_sha_hex = head.commit.hexsha

In [6]:
### Preparation
def resetToHead():
    repo.git.reset('--hard', head_sha_hex) 
resetToHead()

def resetToCommit(commit_id):
    repo.git.reset('--hard', commit_id)

#### Liste mit allen Commits von alt nach neu erstellen

In [7]:
# circa 8 minuten
# global commits
# commits = project.commits.list(all=True, query_parameters={'ref_name': 'master'})
# commits.reverse()

## Datenerhebung

### Funktionen zur Datenverarbeitung
#### Endung, LoC, Couplings

In [8]:
### Used Function in MYSQL Prep
def getEnding(file):
    ende = len(file)
    begin = -1
    for i in reversed(range(len(file))):
        if file[i] == ".":
            begin = i
            break
    if begin != -1:
        ending = file[begin:ende]
    else:
        ending = 0
    return ending

def countLineOfFile(file):
    f = open(file, "r", encoding="utf8")
    line_count = 0
    for line in file:
        if line != "/n":
            line_count += 1
    f.close()
    return line_count

def getCouplings():
    mycursor.execute("SELECT commit_id FROM stats WHERE stats_id=(SELECT MAX(stats_id) FROM stats)")
    count_commits = mycursor.fetchall()[0][0]
    print(count_commits)
    all_couplings = []
    for i in range(count_commits):
        couplings_per_commit = []
        i+=1
        if np.mod(i, 100) == 0:
            print(i)
        mycursor.execute("SELECT file_name FROM stats WHERE commit_id=%s", (str(i), ))
        files_to_commit = mycursor.fetchall()
        for a in range(len(files_to_commit)):
            file1 = files_to_commit[a][0]
            for b in range(a+1, len(files_to_commit)):
                file2 = files_to_commit[b][0]
                couplings_per_commit.append([file1, file2])
        all_couplings.append(couplings_per_commit) 
    return all_couplings



In [9]:
### Functions to preparate Data

## Get Last if no ending
def getLast(path):
    ende = len(path)
    begin = -1
    for i in reversed(range(len(path))):
        if path[i] == "/":
            begin = i
            break
    if begin != -1:
        ending = path[begin:ende] #ending hinzugefügt
    else:
        ending = 0 #keine .endung 
    return ending

# Returns Liste mit CommitInizes welche string stri in message enthalten
def getCommitInidzesWithStringInMessage(stri):
    mycursor.execute("SELECT message FROM commits")
    messages = mycursor.fetchall()
    indizes = []
    for i, message in enumerate(messages):
        i += 1
        if stri.casefold() in message[0].casefold():
            indizes.append(i)
    return indizes 




### Daten erheben und in MySQL Datenbank schreiben

In [10]:
#first global variable !
def insertCommits(): # 10 min
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("ALTER TABLE commits AUTO_INCREMENT = 1")
    mycursor.close() 
    a = 1
    sql = "INSERT INTO commits (hexsha, author, authoredDate, committer, committed_date, message, authoredDate_real, committed_date_real, author_mail, committer_email) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    for i in range(len(commits)):
        if np.mod(a, 100) == 0:
            print(a)
        a+=1
        # print(commit)
        hexsha = commits[i].id
        # print(commit.author.name)
        author = commits[i].author_name
        authored_date = dateutil.parser.parse(commits[i].authored_date)
        committer = commits[i].committer_name
        committed_date = dateutil.parser.parse(commits[i].committed_date)
        message = commits[i].message
        authored_date_real = commits[i].authored_date
        committed_date_real = commits[i].committed_date
        val = (hexsha, author, authored_date, committer, committed_date, message, authored_date_real, committed_date_real, commits[i].author_email, commits[i].committer_email)
        mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
        mycursor = mydb.cursor()
        mycursor.execute(sql,val)
        mydb.commit()
        mycursor.close()   

def insertStats(): # 3/4 h
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()    
    sql = "INSERT INTO stats (commit_id, file_name, insertions, deletions, total) VALUES (%s, %s, %s, %s, %s)"
    liste = list(repo.iter_commits("master"))
    commits = liste[::-1]
    print(len(commits))
    for counter, commit in enumerate(commits):
        val = []
        if np.mod(counter, 100)==0:
            print(counter)
        allStats = commit.stats.files
        # print(allStats.keys())
        dateien = list(allStats.keys())
        values_dicts = list(allStats.values())
        ins_dels=[list(val.values()) for val in values_dicts]
        val = [(counter+1, datei, *in_del) for datei, in_del in zip(tuple(dateien), tuple(ins_dels))]
        mycursor.executemany(sql,val) 
        mydb.commit()

def insertFiles():
    sql = "INSERT INTO files (file_name, ending, init_CLOC) VALUES (%s, %s, %s)"
    commits = list(repo.iter_commits("master"))
    all_files = []
    for i in range(len(commits)):
        i += 1
        if np.mod(i, 100) == 0:
            print(i)
        mycursor.execute("SELECT file_name, insertions FROM stats WHERE commit_id=%s", (i, ))
        stats_to_commit = mycursor.fetchall()
        for item in stats_to_commit:
            if " " in item[0]:
                continue
            elif item[0] not in all_files:
                all_files.append(item[0])
    print("alle files in liste")
    # initial CLOC
    initialCLOC = []
    print("Anzahl alle Files:", len(all_files))
    mycursor.execute("SELECT file_name, insertions FROM stats WHERE commit_id=1")
    initial_files = mycursor.fetchall()
    for item in initial_files:
        initialCLOC.append([item[0], getEnding(item[0]), item[1]])
    print("initiale dateien drinne")
    mycursor.executemany(sql, initialCLOC)
    mydb.commit()
    initialCLOC = []
    for counter, fn in enumerate(all_files):
        if np.mod(counter, 100) == 0:
            mycursor.executemany(sql, initialCLOC)
            mydb.commit()
            initialCLOC = []
        if not any(fn in i for i in initial_files):
            initialCLOC.append([fn, getEnding(fn), 0])
    print("andere sind auch dabei")

def insertCouplings():
    sql = "INSERT INTO couplings (file_name1, file_name2, count) VALUES (%s, %s, %s)"
    compressed = []
    allCouplings = getCouplings()
    print("es gibt ", len(allCouplings), "couplings")
    for index in range(len(allCouplings)):
        for couplingIndex in range(len(allCouplings[index])):
            file1 = allCouplings[index][couplingIndex][0]
            file2 = allCouplings[index][couplingIndex][1]
            found = False
            if len(compressed) != 0:
                for couple in compressed:
                    # print("couple:", couple)
                    if file1 and file2 in couple:
                        couple[2] += 1 
                        found = True
                        break
                if not found:
                    compressed.append([file1, file2, 1])
                    break
            else:
                compressed.append([file1, file2, 1])
    compressed.sort(key=itemgetter(2))
    print("insgesamt", len(compressed), "couplings")
    for i, couple in enumerate(compressed):
        if np.mod(i, 100) == 0:
            print(i)
        val = (couple[:])
        mycursor.execute(sql,val)
        mydb.commit()
    print("couplings in datenbank")
    
def insertIssues():
    sql_closed = "INSERT INTO issues (title, description, state, author, created_at, closed_at, closed_by, labels) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    sql_open = "INSERT INTO issues (title, description, state, author, created_at, labels) VALUES (%s, %s, %s, %s, %s, %s)"
    issues = project.issues.list(all=True)
    print("anzahl issus:", len(issues))
    issues_sorted = sorted(issues, key=operator.attrgetter("created_at"))
    print("issues sortiert")
    for i, issue in enumerate(issues_sorted):
        if np.mod(i,100) == 0:
            print(i)
        title = issue.title
        description = issue.description
        state = issue.state
        author = issue.author['name']
        created_at = datetime.datetime.strptime(str(issue.created_at), "%Y-%m-%dT%H:%M:%S.%fZ")
        labels = issue.labels
        if issue.closed_at != None:
            closed_at = datetime.datetime.strptime(str(issue.closed_at), "%Y-%m-%dT%H:%M:%S.%fZ")
            closed_by = issue.closed_by
            val = (str(title), str(description), str(state), str(author), str(created_at), str(closed_at), str(closed_by), str(labels))
            mycursor.execute(sql_closed,val)
            mydb.commit()
        else:
            val = (str(title), str(description), str(state), str(author), str(created_at), str(labels))
            mycursor.execute(sql_open,val)
            mydb.commit()
    print("Issues inserted")

def insertReleases():
    sql = "INSERT INTO releases (name, description, released_at, author, commit_id) VALUES (%s, %s, %s, %s, %s)"
    releases = project.releases.list(all = True)
    releases.reverse()
    for release in releases:
        name = release.name
        description = release.description
        released_at = datetime.datetime.strptime(str(release.released_at), "%Y-%m-%dT%H:%M:%S.%fZ")
        author = release.author['name']
        commit_id = release.commit['id']
        val = (str(name), str(description), str(released_at), str(author), str(commit_id))
        mycursor.execute(sql, val)
        mydb.commit()
        
def insertLabels():
    mycursor.execute("ALTER TABLE labels AUTO_INCREMENT = 1")
    sql = "INSERT INTO labels (label_id, name, description) VALUES (%s, %s, %s)"
    labels = project.labels.list(all = True)
    for label in labels:
        label_id = label.id
        name = label.name
        description = label.description
        val = (label_id, name, description)
        mycursor.execute(sql, val)
        mydb.commit()

def insertMergeRequests():
    mrs = project.mergerequests.list(all=True)
    mrs.reverse()
    print("all merge requests loaded")
    sql_merged = "INSERT INTO merge_requests (title, description, state, merged_by, merged_at, merge_status) VALUES (%s, %s, %s, %s, %s, %s)"
    sql_merged_not = "INSERT INTO merge_requests (title, description, state, merge_status) VALUES (%s, %s, %s, %s)"
    for mr in mrs:
        title = mr.title
        description = mr.description
        state = mr.state
        merge_status = mr.merge_status
        if mr.merged_by != None and mr.merged_at != None:
            merged_by = mr.merged_by['name']
            merged_at = datetime.datetime.strptime(str(mr.merged_at), "%Y-%m-%dT%H:%M:%S.%fZ")
            val = (str(title), str(description), str(state), str(merged_by), str(merged_at), str(merge_status))
            mycursor.execute(sql_merged, val)
            mydb.commit()
        else: 
            val = (str(title), str(description), str(state), str(merge_status))
            mycursor.execute(sql_merged_not, val)
            mydb.commit()
        

In [11]:
def initialInsertions(): #takes long time - really
    insertCommits()
    insertStats() 
    insertFiles()
    insertCouplings()
    insertIssues()
    insertReleases()
    insertLabels()
    insertMergeRequests()

# initialInsertions() # only when new repo 

### Softwareanalyse durchführen und Ergebnisse in MySQL Datenbank schreiben

#### Cppcheck

In [12]:
## CODE ANALYSIS cppchecker
data_place = "E:/Bachelor/Gitlab_Inkscape/XML_CPP_Checker/"

def getAndSafe(commit_id, file):
    # print(file)
    call = subprocess.getoutput(['cppcheck', '--enable=all', '--template={file},{line},{severity},{id},{message}', file])
    for line in call.splitlines():
        x = line.split(',')
        if len(x) == 5:  
            insert = (str(commit_id), )+tuple(x)
            lst.append(insert)
    return 
    
def insertCCPCheck(commit_id, files):
    threads = []
    global lst
    lst = []
    for item in files:
        t = threading.Thread(target=getAndSafe, args=(commit_id, item, ))
        threads.append(t)
        t.start()
        # inserts = getAndSafe(i, item)
        # print(inserts)
    for thread in threads:
        thread.join()
    # print("------", lst)
    mycursor.executemany("INSERT INTO cppcheck (commit_id, file_name, line, categorie, fehler, msg) VALUES (%s, %s, %s, %s, %s, %s)", lst)
    mydb.commit()
    print("----------- Anzahl überprüfter Files: ", len(files))

def cppChecker():
    mycursor.execute("SELECT MAX(id) FROM commits")
    anzahl_commits = mycursor.fetchall()[0][0]
    sql_hexsha = "SELECT hexsha FROM commits WHERE id=%s"
    sql = "SELECT file_name FROM stats WHERE commit_id=%s"
    # for i in range(anzahl_commits+1):
    for i in range(1392, anzahl_commits):
    # for i in range(10,13):
        i += 1
        count_files = 0
        #reset repo to commit
        mycursor.execute(sql_hexsha, (i, ))
        hexsha = mycursor.fetchall()
        resetToCommit(hexsha)
        print("reseted to commit: ", str(i))
        #select files changed in commit
        mycursor.execute(sql, (i, ))
        files_per_commit = mycursor.fetchall()
        print("Anzahl geänderter Files in Commit ",str(i), ":", str(len(files_per_commit)))
        parallele = []
        for j, item in enumerate(files_per_commit):
            # check weather ending is .cpp, .c or .h
            ending = getEnding(item[0])
            if ending == ".cpp" or ending == ".c" or ending == ".h":
                file_path = Path(repo_path_local+item[0])
                try:
                    if file_path.is_file():
                        parallele.append(file_path)
                        count_files += 1
                except:
                    continue
            if np.mod(j,30) == 0 and parallele != []:
                insertCCPCheck(i, parallele)
                parallele = []
        insertCCPCheck(i,parallele)
        print("Anzahl überprüfter Files im commit: ", count_files)

# cppChecker() #2000 min

#### Flawfinder

In [13]:
## CODE ANALYSIS flawfinder
def flawFinder():
    # mycursor.execute("DELETE FROM flawfinder")
    # mycursor.execute("ALTER TABLE flawfinder AUTO_INCREMENT = 1")
    # analyse changed path after every commit
    mycursor.execute("SELECT MAX(id) FROM commits")
    anzahl_commits = mycursor.fetchall()[0][0]
    sql_hexsha = "SELECT hexsha FROM commits WHERE id=%s"
    sql = "SELECT file_name FROM stats WHERE commit_id=%s"
    saveFolder = "E:/Bachelor/Gitlab_Inkscape/flawFinder/commit"
    for i in range(anzahl_commits+1):
        i += 1
        count_files = 0
        #reset repo to commit
        mycursor.execute(sql_hexsha, (i, ))
        hexsha = mycursor.fetchall()
        resetToCommit(hexsha)
        print("reseted to commit: ", str(i))
        #select files changed in commit
        mycursor.execute(sql, (i, ))
        files_per_commit = mycursor.fetchall()
        print("Anzahl geänderter Files in Commit ",str(i), ":", str(len(files_per_commit)))
        for j, item in enumerate(files_per_commit):
            if np.mod(j,100) == 0:
                print(j)
            # check weather ending is .cpp, .c or .h
            ending = getEnding(item[0])
            if ending == ".cpp" or ending == ".c" or ending == ".h":
                file_path = Path(repo_path_local+item[0])
                try:
                    if file_path.is_file():
                        liste = []
                        call = subprocess.getoutput(['flawfinder', '--csv', file_path])
                        count_files += 1
                        for line in call.splitlines():
                            x = line.split(',')
                            if len(x) == 19:  
                                del x[8:]
                                del x[2]
                                insert = (str(i), )+tuple(x)
                                liste.append(insert)
                        mycursor.executemany("INSERT INTO flawfinder (commit_id, file_name, line, DefaultLevel, Level, categorie, name, msg) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", liste)
                        mydb.commit()
                except:
                    continue
        print("Anzahl überprüfter Files: ", count_files)

    # flawfinder --csv --context C:\Users\Johanna\Documents\second_inkscape\inkscape\src\io  > C:\Users\Johanna\Documents\second_inkscape\results2.txtn  

# flawFinder() #800 min


## Datenvorbereitung
### Dateien anhand der Endung kategorisieren

In [14]:
def categoriesForFiles(): #manuell für jedes Repo durchzuführen, da weitere Dateitypen genutzt werden könnten
    #per Ending
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("UPDATE files SET categorie='Sprache' WHERE ending IN ('.po', '.in')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Quellcode' WHERE ending IN ('.cpp', '.c', '.h', '.mingw', '.rc', '.py', '.cmake', '.nsh', '.pod', '.sh', '.html', '.bat', '.yml', '.yaml', '.wxs', '.m4', '.java', '.patch', '.pm', '.osx', '.js', '.spec', '.tcl', '.scpt', '.hxx', '.project', '.jar', '.dll', '.modules', '.mo', '.ps', '.emf', '.json', '.hpp', '.S', '.cmd', '.rb', '.db', '.pem')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Grafik' WHERE ending IN ('.svg', '.pot', '.inx', '.ico', '.xsl', '.png', '.glade', '.css', '.pl', '.0/gtkrc', '.icns', '.theme', '.ds_store', '.svgz', '.xaml', '.bmp', '.jpg', '.ttf', '.eps', '.dtd', '.ink', '.David_icons')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Konfiguration' WHERE ending IN ('.ac', '.am', '.nsi', '.ui', '.its', '.dep', '.plist', '.conf', '.ini', '.ui', '.tst', '.pbxproj', '.exp', '.tpl', '/Makefile_insert')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Text' WHERE ending IN ('.txt', '.xml', '.md', '.bundle', '.dox', '.README', '.xslt', '.otf', '.template', '.dox', '.rtf', '.pdf', '.dict', 'AUTHORS', 'TRANSLATORS')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Sonstiges' WHERE ending IN ('.91','.skip', '.gpl', '.list', '.xpm', '.common', '.desktop', '.ofiles', '.build', '.files', '.meson', '.exclude', '.rl', '.outline', '.include', '.orig', '.rej', '.LGPL', '.gdl-dock', '.cproject', '.test', '.obj', '.uninstall', '.cdr', '.loc', '.supp', '.THIS' ,'.moved', '.ed', '.DS_Store', '.dirstamp', '.icon_themes', '.old', '.ods')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Versionskontrolle' WHERE ending IN ('.diff', '/ChangeLog', '.gitignore', '.gitmodules', '.cvsignore', '.bzrignore')")
    mydb.commit()
    mycursor.execute("UPDATE files SET categorie='Repo_Konfig' WHERE ending IN ('0')")
    mydb.commit()

# categoriesForFiles()



### Label kategorisieren

In [15]:
# manuell für jedes Repo durchzuführen, da Labels individuell für Projekte angelegt werden (können)
def categoriesLabel():
    labels = [["Kritsch"], ["Funktionalität \n eingeschränkt"], ["Abgeschlossen"], ["Dialog"], ["Code Problem"], ["Design"], ["Dokumentation"], ["Betriebssysteme"], ["Tool"], ["Review"], ["Input"], ["Backport"], ["Versionen"], ["Projekte"], ["Sprache"], ["Sonstiges"],["Kein Label"]]
    categories = [["Blocker", "Importance::High", "Importance::Critical"]]
    categories.append(['Importance::Low', 'Importance::Medium', 'bug', 'crash', 'easy-fix', 'incident', 'upstream bug', 'performance', 'preferences', 'To Do', 'refactoring', 'internal extensions', 'improvement', 'building'])
    categories.append(['Closed::Expected Behavior', 'Closed::Not Inkscape', 'Closed::Out of scope'])
    categories.append(['Dialog::Document Properties', 'Dialog::Align & Distribute', 'Dialog::Arrange', 'Dialog::Export PNG', 'Dialog::Fill & Stroke', 'Dialog::Filters', 'Dialog::Find and Replace', 'Dialog::Icon Preview', 'Dialog::LPE', 'Dialog::Layers', 'Dialog::Object properties', 'Dialog::Objects', 'Dialog::Paint servers', 'Dialog::Preferences', 'Dialog::SVG Font Editor', 'Dialog::Selectors and CSS', 'Dialog::Swatches', 'Dialog::Symbols', 'Dialog::Text and Font', 'Dialog::Trace Bitmap', 'Dialog::Transform', 'Dialog::Welcome', 'Dialog::XML editor', 'Dialogs', 'filters'])
    categories.append(['Code location identified', 'Commit identified', 'Boolean operations', 'command line interface (CLI)', 'live path effects (LPE)', 'regression', 'save+export', 'Testing'])
    categories.append(['CSS', 'UI', 'UX', 'UX approved', 'Markers', 'Menus', 'rendering', 'Grids', 'canvas',])
    categories.append(['Documentation', 'Request for comments', 'New feature', 'To Forecast'])
    categories.append(['OS::Linux', 'OS::Windows', 'OS::macOS'])
    categories.append(['Snapping', 'Tool::3D Box', 'Tool::Bezier', 'Tool::Calligraphy','Tool::Color picker', 'Tool::Connectors', 'Tool::Ellipses', 'Tool::Eraser', 'Tool::Fill', 'Tool::Freehand lines', 'Tool::Gradients', 'Tool::Measurements', 'Tool::Mesh Gradient', 'Tool::Node', 'Tool::Rectangles', 'Tool::Select','Tool::Spray', 'Tool::Stars', 'Tool::Text', 'Tool::Tweak', 'Tool::Zoom'])
    categories.append(['needs design', 'needs info', 'needs investigation', 'needs reproducibility', 'needs testing', 'guides', 'help wanted'])
    categories.append(['Input::Keyboard', 'Input::Shortcuts', 'Input::Touchpad', 'Input::tablet'])
    categories.append(['backport::done', 'backport::proposed', 'backport::rejected'])
    categories.append(['open+import', 'packaging', 'packaging::AppImage', 'packaging::snap', 'patch', 'undo / redo', 'meta', 'release management', 'Doing', 'Next Version Feature'])
    categories.append(['Project::Command palette', 'Project::GDL replacement', 'Project::Multipage', 'Project::SVG2'])
    categories.append(['verbs / actions', 'translations', 'text'])
    categories.append(['theming', 'Presets - Content', 'GTK 3', 'Gtk 4', 'Wayland', 'livarot'])
    categories.append([''])
    return labels, categories
# categoriesLabel()
def deep_index(lst, w):
    return [(i, sub.index(w)) for (i, sub) in enumerate(lst) if w in sub]



In [16]:
#reihenfolge bahängig von commitzeitpunkt
def commit_time_folge():
    id_c_folge = []
    id_c_real_folge = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select id from commits
    order by committed_date""")
    for (c_id, ) in mycursor:
        id_c_folge.append(c_id)
    mycursor.close()
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select id from commits
    order by committed_date_real""")
    for (c_id, ) in mycursor:
        id_c_real_folge.append(c_id)
    mycursor.close()

    for i in range(len(id_c_folge)-1):
        if id_c_folge[i] != id_c_folge[i+1]-1:
            print("ID1:", id_c_folge[i], "folgt auf ID2:", id_c_folge[i+1])
    print("------------------------------------------------------------------------")
    a = 0
    for i in range(len(id_c_real_folge)-1):
        if id_c_real_folge[i] != id_c_real_folge[i+1]-1:
            print("ID1:", id_c_real_folge[i], "folgt auf ID2:", id_c_real_folge[i+1])
            a+=1
    print("Die Reiehnfolge abhängig der Commitzeitpunkte Serverzeit und Lokale Uhrzeit weichen in",a,"Fällen voneinander ab.")

## Daten visualisieren --> Fragestellungen beantworten

### Frage 1: Wie viele Commits werden im Branch „master“ getätigt?

len(commits) = 25569 und davon dann die gemergden über kommandozeilenbefehl ("git rev-list --no-merges --count HEAD")

In [17]:
def frage1():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(id) from commits''')
    for (count, ) in mycursor:
        count_commits = count
    mycursor.close()
    print("Es wurden", count_commits, "Commits im Master Branch getätigt")

# frage1()


### Frage 2: Wie viele Autoren / Committer haben am Projekt „Inkscape“ gearbeitet?

In [18]:
def frage2():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(distinct author),count(distinct committer)  from commits''')
    for a, c in mycursor:
        a_count = a
        c_count = c
    mycursor.close()
    print("Es haben", a_count, "Autoren und", c_count, "Committer am Projekt gearbeitet.")
    x_anzahl_commits_committer = []
    y_anzahl_committer = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select num, count(committer) from
        (select committer, count(*) as num 
        from commits
        group by committer) as t
        group by num''')
    for num, count in mycursor:
        x_anzahl_commits_committer.append(num)
        y_anzahl_committer.append(count)
    mycursor.close()

    x_anzahl_commits_author = []
    y_anzahl_author = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select num, count(author) from
        (select author, count(*) as num 
        from commits
        group by author) as t
        group by num''')
    for num, count in mycursor:
        x_anzahl_commits_author.append(num)
        y_anzahl_author.append(count)
    mycursor.close()
    # print(len(x_anzahl_commits_committer),len(y_anzahl_committer), len(x_anzahl_commits_author), len(y_anzahl_author) )
    fig, ax = plt.subplots()
    #ax.set_xticks(np.linspace(0,30000,7))
    #ax.set_xticklabels([0, 5000, 10000, 15000, 20000, 25000, 30000])
    fig.set_size_inches(30,15)
    plt.subplot(2,1,1)
    plt.xlabel("Anzahl Commits")
    plt.ylabel("Anzahl Committer")
    plt.title("Verteilung Anzahl Committer - Anzahl Commits")
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    plt.bar(x_anzahl_commits_committer, y_anzahl_committer, width=1, color="blue", align="center")

    plt.subplot(2,1,2)
    plt.xlabel("Anzahl Änderungen")
    plt.ylabel("Anzahl Autoren")
    plt.title("Verteilung Anzahl Autoren - Anzahl Commits")
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    plt.bar(x_anzahl_commits_author, y_anzahl_author, width=1, color="red", align="center")
    plt.savefig("FRAGE_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

# frage2()

### Frage 3: Wie viele Releases werden veröffentlicht?
Anmerkung: seit Anfang 2020 Releasmanagement auf gitlab, was davor geschah ist der Projektdoku zu entnehmen


In [19]:
def frage3():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(id) from releases''')
    for (a,) in mycursor:
        r_count = a
    mycursor.close()
    print("Seit Beginn 2020 wurden", r_count,"Releases veröffentlicht.")

# frage3()

### Frage 4: Wie viele Zeilen werden verändert?

In [20]:
def frage4():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select sum(insertions)+sum(deletions), sum(insertions), sum(deletions) from stats''')
    for total_sql, ins_sql, dels_sql in mycursor:
        total = total_sql
        ins = ins_sql
        dels = dels_sql
    mycursor.close()
    print("Insgesamt: ", total)
    print("Hinzugefügt: ", ins)
    print("Gelöscht: ", dels)
    count_commits_author = []
    count_ins_author = []
    count_dels_author = []
    author_name = []

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select count(distinct commit_id) as coun, sum(insertions), sum(deletions), author from
        (select commit_id, insertions, deletions, author, committer from stats
        join commits
        where stats.commit_id=commits.id) as t
        group by author
        order by coun desc""")
    for count_commits, count_ins, count_dels, name_cur in mycursor:
        count_commits_author.append(count_commits)
        count_ins_author.append(count_ins)
        count_dels_author.append(count_dels)
        author_name.append(name_cur)
    mycursor.close()
    # print(len(count_commits_author),len(count_ins_author),len(count_dels_author),len(author_name))

    count_commits_committer = []
    count_ins_committer = []
    count_dels_committer = []
    committer_name = []

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select count(distinct commit_id) as coun, sum(insertions), sum(deletions), committer from
        (select commit_id, insertions, deletions, author, committer from stats
        join commits
        where stats.commit_id=commits.id) as t
        group by committer
        order by coun desc""")
    for count_commits, count_ins, count_dels, name_cur in mycursor:
        count_commits_committer.append(count_commits)
        count_ins_committer.append(count_ins)
        count_dels_committer.append(count_dels)
        committer_name.append(name_cur)
    mycursor.close()
    # print(len(count_commits_committer),len(count_ins_committer),len(count_dels_committer),len(committer_name))
    fig, ax = plt.subplots()
    #ax.set_xticks(np.linspace(0,30000,7))
    #ax.set_xticklabels([0, 5000, 10000, 15000, 20000, 25000, 30000])
    fig.set_size_inches(22,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Name Autor", fontsize=21,fontweight='bold')
    plt.xticks( np.arange(len(author_name)), author_name, rotation=90, fontsize=4)
    plt.ylabel("Anzahl Zeilen", fontsize=21,fontweight='bold')
    plt.title("Anzahl hinzugefügte und gelöschte Zeilen pro Autor", fontsize=23,fontweight='bold')
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    # plt.bar(np.linspace(1,len(author_name),len(author_name)), count_commits_author, width=0.4, color="black", align="center")
    plt.bar(np.linspace(0,len(author_name)-1,len(author_name))-0.2, count_ins_author, width=0.4, color="green", align="center")
    plt.bar(np.linspace(0,len(author_name)-1,len(author_name))+0.2, count_dels_author, width=0.4, color="red", align="center")
    plt.savefig("FRAGE_4_1_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots()
    #ax.set_xticks(np.linspace(0,30000,7))
    #ax.set_xticklabels([0, 5000, 10000, 15000, 20000, 25000, 30000])
    fig.set_size_inches(22,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Name Committer", fontsize=21,fontweight='bold')
    plt.xticks( np.arange(len(committer_name)), committer_name, rotation=90, fontsize=4)
    plt.ylabel("Anzahl Zeilen", fontsize=21,fontweight='bold')
    plt.title("Anzahl hinzugefügte und gelöschte Zeilen pro Committer", fontsize=23,fontweight='bold')
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    # plt.bar(np.linspace(1,len(committer_name),len(committer_name)), count_commits_committer, width=0.4, color="black", align="center")
    plt.bar(np.linspace(0,len(committer_name)-1,len(committer_name))-0.2, count_ins_committer, width=0.4, color="green", align="center")
    plt.bar(np.linspace(0,len(committer_name)-1,len(committer_name))+0.2, count_dels_committer, width=0.4, color="red", align="center")
    plt.savefig("FRAGE_4_1_2.png",dpi=240, bbox_inches = 'tight')
    plt.subplots_adjust(left=0.125,
                    bottom=0.05, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.35)
    plt.show()

    x = np.linspace(0,2,3)
    y_in = [3193007, 196281, 372055]
    y_del = [1719922, 104320, 170357]

    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,2,3))
    ax.set_xticklabels(["Quelltext", "Kommentare", "Leerzeilen"], fontsize=29)
    fig.set_size_inches(30,15)
    plt.xlabel("Kategorie", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Zeilen", fontsize=35, fontweight='bold')
    # plt.title("Anzahl Zeilen pro Kategorie", fontsize=23, fontweight='bold')
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    plt.bar(x-0.2, y_in, width=0.4, color="green", align="center")
    plt.bar(x+0.2, y_del, width=0.4, color="red", align="center")

    plt.savefig("FRAGE_4_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

# frage4()

### Frage 5: Wie viele Fehler werden mit den genutzten Analyse-Tools gefunden?

In [21]:
def frage5():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(id) from cppcheck''')
    for (count,) in mycursor:
        print("Insgesamt", count ,"Meldungen von CppCheck")
    mycursor.close()
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(id) from flawfinder''')
    for (count,) in mycursor:
        print("Insgesamt", count ,"potentielle Schwachstellen von Flawfinder")
    mycursor.close()
    
    werte = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(kennung) as count, kennung from cppcheck 
        group by kennung
        order by kennung''')
    for count, kennung in mycursor:
        werte[str(kennung)] = count
    mycursor.close()
    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,5,6))
    ax.set_xticklabels(["Error", "Information", "Performanz", "Portabilität", "Stil", "Warnung"], fontsize=40)
    fig.set_size_inches(30,15)
    plt.xlabel("Kategorien \n unterteilt in Meldungen", fontsize=50,fontweight='bold')
    plt.ylabel("Anzahl Meldungen", fontsize=50,fontweight='bold')
    # plt.title("Verteilung der Meldungen durch CppCheck", fontsize=20)
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    bottom_bar0 = 0
    bottom_bar1 = 0
    bottom_bar2 = 0
    bottom_bar3 = 0
    bottom_bar4 = 0
    bottom_bar5 = 0
    i_0 = 0
    i_1 = 0
    i_2 = 0
    i_3 = 0
    i_4 = 0
    i_5 = 0
    colors_0 = mcp.gen_color(cmap="plasma",n=40)
    colors_1 = mcp.gen_color(cmap="plasma",n=7)
    colors_2 = mcp.gen_color(cmap="plasma",n=7)
    colors_3 = mcp.gen_color(cmap="plasma",n=10)
    colors_4 = mcp.gen_color(cmap="plasma",n=54)
    legend0 = []
    legend1 = []
    legend2 = []
    legend3 = []
    legend4 = []
    legend5 = []
    werte_0 = []
    werte_1 = []
    werte_2 = []
    werte_3 = []
    werte_4 = []
    werte_5 = []
    colors_5 = mcp.gen_color(cmap="plasma",n=49)
    for wert in werte:
        try:
            if int(wert) < 200:
                l1, = plt.bar(0, werte[wert], bottom=bottom_bar0, label="Error"+str(wert), color=colors_0[i_0])
                werte_0.append(wert)
                legend0.append(l1)
                i_0 +=1
                bottom_bar0 += werte[wert]
            if int(wert) > 200 and int(wert) < 300:
                l1, = plt.bar(1, werte[wert], bottom=bottom_bar1, label="Info"+str(wert), color=colors_1[i_1])
                werte_1.append(wert)
                legend1.append(l1)
                i_1 +=1
                bottom_bar1 += werte[wert]
            if int(wert) > 300 and int(wert) < 400:
                l1, = plt.bar(2, werte[wert], bottom=bottom_bar2, label="Performanz"+str(wert), color=colors_2[i_2])
                werte_2.append(wert)
                legend2.append(l1)
                i_2 +=1
                bottom_bar2 += werte[wert]
            if int(wert) > 400 and int(wert) < 500:
                l1, = plt.bar(3, werte[wert], bottom=bottom_bar3, label="Portabilität"+str(wert), color=colors_3[i_3])
                werte_3.append(wert)
                legend3.append(l1)
                i_3 +=1
                bottom_bar3 += werte[wert]
            if int(wert) > 500 and int(wert) < 600:
                l1, = plt.bar(4, werte[wert], bottom=bottom_bar4, label="Stil"+str(wert), color=colors_4[i_4])
                werte_4.append(wert)
                legend4.append(l1)
                i_4 +=1
                bottom_bar4 += werte[wert]
            if int(wert) > 600 :
                l1, = plt.bar(5, werte[wert], bottom=bottom_bar5, label="Warnung"+str(wert), color=colors_5[i_5])
                werte_5.append(wert)
                legend5.append(l1)
                i_5 +=1
                bottom_bar5 += werte[wert]
        except:
            continue
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()
    legend_0 = ax.legend(legend0, werte_0, title="Error", loc=2, bbox_to_anchor=(0,1), fontsize=10)
    legend_1 = ax2.legend(legend1, werte_1, title="Information", loc=2, bbox_to_anchor=(0.035,1), fontsize=10)
    legend_2 = ax3.legend(legend2, werte_2, title="Performanz", loc=2, bbox_to_anchor=(0.076,1), fontsize=10)
    legend_3 = ax4.legend(legend3, werte_3, title="Portabilität", loc=2, bbox_to_anchor=(0.117,1), fontsize=10)
    legend_4 = ax5.legend(legend4, werte_4, title="Stil", loc=2, bbox_to_anchor=(0.157,1), fontsize=10)
    legend_5 = ax6.legend(legend5, werte_5, title="Warnung", loc=2, bbox_to_anchor=(0.193,1), fontsize=10)

    # plt.legend(loc='upper left', ncol=5, title="Kennungen der Meldungen")
    plt.savefig("FRAGE_5_1.png",dpi=240, bbox_inches = 'tight')

    plt.show()

    werte = {"buffer": {}, "format":{}, "integer": {}, "tmpfile":{}}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(kennung) as count, name, categorie from flawfinder 
        group by kennung
        order by kennung''')
    for count, name_sql, categorie in mycursor:
        werte[str(categorie)][str(name_sql)] = count
    mycursor.close()

    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,3,4))
    ax.set_xticklabels(["Buffer", "Format", "Integer", "TmpFile"], fontsize=40)
    fig.set_size_inches(30,15)
    plt.xlabel("Potentialle Schwachstellen \n unterteilt in Kategorien und Meldungen", fontsize=50,fontweight='bold')
    plt.ylabel("Anzahl Meldungen", fontsize=50,fontweight='bold')
    # plt.title("Verteilung der Meldungen durch Flawfinder", fontsize=20)
    # plt.xlim(0,1600)
    # plt.ylim(0,250)
    bottom_bar0 = 0
    i = 0
    colors_0 = [mcp.gen_color(cmap="plasma",n=20), mcp.gen_color(cmap="plasma",n=20), mcp.gen_color(cmap="plasma",n=20), mcp.gen_color(cmap="plasma",n=20)]
    legend0 = []
    legend1 = []
    legend2 = []
    legend3 = []
    werte_0 = []
    werte_1 = []
    werte_2 = []
    werte_3 = []
    for categorie in werte:
        a = 0
        for name_sql in werte[categorie]:
            l1, = plt.bar(i, werte[categorie][name_sql], bottom=bottom_bar0, label=str(name_sql), color=colors_0[i][a])
            a+=1
            bottom_bar0 += werte[categorie][name_sql]
            if i == 0:
                legend0.append(l1)
                werte_0.append(str(name_sql))
            elif i == 1:
                legend1.append(l1)
                werte_1.append(str(name_sql))
            elif i == 2:
                legend2.append(l1)
                werte_2.append(str(name_sql))
            elif i == 3:
                legend3.append(l1)
                werte_3.append(str(name_sql))
        i += 1
        bottom_bar0 = 0
    
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    legend_0 = ax.legend(legend0[::-1], werte_0[::-1], title="Buffer", loc=1, bbox_to_anchor=(0.854,1))
    legend_1 = ax2.legend(legend1[::-1], werte_1[::-1], title="Format", loc=1, bbox_to_anchor=(0.901,1))
    legend_2 = ax3.legend(legend2[::-1], werte_2[::-1], title="Integer", loc=1, bbox_to_anchor=(0.938,1))
    legend_3 = ax4.legend(legend3[::-1], werte_3[::-1], title="TmpFile", loc=1, bbox_to_anchor=(0.99,1))
    # plt.legend(loc='upper right', ncol=5)
    plt.savefig("FRAGE_5_2.png",dpi=240, bbox_inches = 'tight')

    plt.show()


def visual_ins_dels_commiter():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select committer, insertions, deletions 
        FROM commits as c 
        join stats as s 
        on c.id = s.commit_id+1""")
    all = {}
    committer_lst = []
    ins = []
    dels = []
    for committer, insertions, deletions in mycursor:
        if committer in all.keys():
            all[committer]["insertions"] += insertions
            all[committer]["deletions"] += deletions
            all[committer]["total"] += deletions+insertions
        else:
            all[committer] = {"insertions":insertions, "deletions":deletions, "total":deletions+insertions}
    mycursor.close()
    all_sorted = dict(sorted(all.items(), key=lambda item: item[1]["total"],reverse=True))
    for name_commiter in all_sorted:
        committer_lst.append(name_commiter)
    for data in all_sorted.values():
        ins.append(data["insertions"])
        dels.append(data["deletions"])
    fig, ax = plt.subplots(dpi=240) 
    fig.set_size_inches(30,15)
    plt.bar([i-0.2 for i in range(len(committer_lst))], ins, color="green", align='center', width=0.4)
    plt.bar([i+0.2 for i in range(len(committer_lst))], dels, color="red", align='center', width=0.4)

    plt.xticks( np.arange(len(committer_lst)), committer_lst, rotation=90, fontsize=4)
    # plt.savefig("E:/Bachelor/Gitlab_Inkscape/visualisierung/ins_dels_per_committer.png",dpi=240, bbox_inches = 'tight')
    plt.ylabel("Anzahl Zeilen", fontsize=40)
    plt.xlabel("Name der Committer", fontsize=40)
    locs,labels = plt.yticks()
    plt.yticks(locs, map(lambda x: "%d" % x, locs))
    plt.show()

def visual_ins_dels_author():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("""select author, insertions, deletions 
        FROM commits as c 
        join stats as s 
        on c.id = s.commit_id+1""")
    all = {}
    committer_lst = []
    ins = []
    dels = []
    for committer, insertions, deletions in mycursor:
        if committer in all.keys():
            all[committer]["insertions"] += insertions
            all[committer]["deletions"] += deletions
            all[committer]["total"] += deletions+insertions
        else:
            all[committer] = {"insertions":insertions, "deletions":deletions, "total":deletions+insertions}
    mycursor.close()
    all_sorted = dict(sorted(all.items(), key=lambda item: item[1]["total"],reverse=True))
    for name_commiter in all_sorted:
        committer_lst.append(name_commiter)
    for data in all_sorted.values():
        ins.append(data["insertions"])
        dels.append(data["deletions"])
    fig, ax = plt.subplots(dpi=240) 
    fig.set_size_inches(30,15)
    plt.bar([i-0.2 for i in range(len(committer_lst))], ins, color="green", align='center', width=0.4)
    plt.bar([i+0.2 for i in range(len(committer_lst))], dels, color="red", align='center', width=0.4)

    plt.xticks( np.arange(len(committer_lst)), committer_lst, rotation=90, fontsize=4)
    # plt.savefig("E:/Bachelor/Gitlab_Inkscape/visualisierung/ins_dels_per_committer.png",dpi=240)
    plt.ylabel("Anzahl Zeilen", fontsize=40)
    plt.xlabel("Name der Antoren", fontsize=40)
    locs,labels = plt.yticks()
    plt.yticks(locs, map(lambda x: "%d" % x, locs))
    plt.show()

# visual_ins_dels_author()
# visual_ins_dels_commiter()
# frage5()

### Frage 6: Wie viele Tickets werden geöffnet und wie lange dauert es bis sie geschlossen werden?

In [22]:
def frage6():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(id) from issues''')
    for (count,) in mycursor:
        print("Insgesamt", count ,"Tickets")
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(id) from issues
        where state="opened"''')
    for (count,) in mycursor:
        print("Insgesamt", count ,"geöffnete Tickets")
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(id) from issues
        where state="closed"''')
    for (count,) in mycursor:
        print("Insgesamt", count ,"geschlossene Tickets")
    mycursor.close()

    categories, labels  = categoriesLabel()
    i=0
    y = [[]]*len(categories)
    points_count = 0
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute("select TIMESTAMPDIFF(second, created_at, closed_at), labels from issues where state='closed'")
    colors = [[]]*len(categories)
    color = ["red", "blue", "green", "pink", "yellow", "orange", "gray", "brown", "lime", "darkmagenta", "cyan", "darkred", "lightgreen", "royalblue", "indigo", "crimson", "magenta"]
    for timediff, labels_sql in mycursor:
        labels_sql = eval(labels_sql)
        if len(labels_sql) == 0:
            index = -1
            colors[index] = colors[index] + [color[index]]
            y[index] = y[index]+ [int(timediff)/3600]
        for label in list(labels_sql):
            index = next(i for i,v in enumerate(labels) if label in v)
            colors[index] = colors[index] + [color[index]]
            y[index] = y[index]+ [int(timediff)/3600]
    mycursor.close()
    
    countii = [0]*len(categories)
    y_open = [[]]*len(categories)
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()

    mycursor.execute("select labels from issues where state='opened'")
    for (labels_sql,) in mycursor:
        labels_sql = eval(labels_sql)
        if len(labels_sql) == 0:
            index = -1
            countii[index] += 1
            y_open[index] = y_open[index]+[countii[index]]
        for label in list(labels_sql):
            index = next(i for i,v in enumerate(labels) if label in v)
            countii[index] += 1
            y_open[index] = y_open[index]+[countii[index]]
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    countii = [0]*len(categories)
    y_close = [[]]*len(categories)
    mycursor.execute("select labels from issues where state='closed'")
    for (labels_sql,) in mycursor:
        labels_sql = eval(labels_sql)
        if len(labels_sql) == 0:
            index = -1
            countii[index] += 1
            y_close[index] = y_close[index]+[countii[index]]
        for label in list(labels_sql):
            index = next(i for i,v in enumerate(labels) if label in v)
            countii[index] += 1
            y_close[index] = y_close[index]+[countii[index]]
    mycursor.close()

    fig, ax = plt.subplots() 
    fig.set_size_inches(22,12)
    ax.ticklabel_format(useOffset=False, style='plain')    
    ax2 = ax.twinx()
    
    ax.set_xticks(np.linspace(1,len(categories), len(categories)))
    ax.set_xticklabels([categorie[0] for categorie in categories], rotation=30)
    ax.set_xlabel("Labelkategorie", fontsize=18,fontweight='bold')

    
    ax.set_ylabel("Zeit Öffnung bis Schließung", fontsize=18,fontweight='bold')
    ax.set_yticks([168, 730, 730*2, 730*3, 730*4, 730*5, 730*6, 730*7, 730*8, 730*9, 730*10, 730*11, 8760, 8760*2, 8760*3, 8760*4])
    ax.set_yticklabels(["1 Woche", "1 Monat", "2 Monate", "3 Monate", "4 Monate", "5 Monate", "6 Monate", "7 Monate", "8 Monate", "9 Monate", "10 Monate", "11 Monate", "1 Jahr", "2 Jahre", "3 Jahre", "4 Jahre"])
    ax2.set_ylabel("Anzahl offener (schwarz) und geschlossener (grau) Tickets", fontsize=18,fontweight='bold')
    ax.tick_params(axis='both', labelsize=10)
    ax.set_ylim(0,21900)
    ax2.set_ylim(0,5000)
    
    # for i in range(len(y_open)):
    #     try:
    #         print(categories[i],":", max(y_open[i])-max(y_close[i]))
    #     except:
    #         continue

    # plt.xlabel("Label Kategorien", fontsize=18)
    # plt.title("Dauer der Öffnung der Tickets in Stunden")
    x = list(np.linspace(1,len(categories), len(categories)))
    for xe, ye, yi, ya, c in zip(x, y, y_open, y_close, colors):
        ax.scatter([xe-0.1] * len(ye), ye, color=c, alpha = 0.1) 
        ax2.scatter([xe+0.1] * len(yi), yi, color="black") 
        ax2.scatter([xe+0.2] * len(ya), ya, color="gray") 
    plt.savefig("FRAGE_6.png",dpi=240, bbox_inches = 'tight')
    plt.show()

# frage6()

### Frage 7: Wann werden die Commits getätigt?

In [25]:
def frage7():
    jahre = {} 
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(id), year(committed_date_real) as y, month(committed_date_real) as m
        from commits 
        group by y, m''')
    for count, jahr, monat in mycursor:
        if str(jahr) not in jahre.keys():
            jahre[str(jahr)] = {}
        jahre[str(jahr)][str(monat)] = count
    mycursor.close()
    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,len(jahre)-1,len(jahre)))
    ax.set_xticklabels([jahr for jahr in jahre])
    fig.set_size_inches(30,15)
    plt.xlabel("Jahr", fontsize=25,fontweight='bold')
    plt.ylabel("Anzahl Commits", fontsize=25,fontweight='bold')
    # plt.title("Verteilung der Commits über die Jahre")
    for jahr in jahre:
        colors = iter([plt.cm.Paired(i) for i in range(12)])
        bottom_bar = 0
        for value in jahre[jahr]:
            plt.bar(jahr, jahre[jahr][value], bottom=bottom_bar, label=calendar.month_name[int(value)] , color=[next(colors)])
            bottom_bar += jahre[jahr][value]
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    values = [val for val in by_label.values()]
    keys = [lab for lab in by_label.keys()]
    plt.legend(values[::-1], keys[::-1], fontsize=18)
    # plt.legend(loc='upper left', ncol=5)
    plt.savefig("FRAGE_7_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()


    wochentage_lst = ['Sonntag', 'Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag']
    wochentage = {"Montag": {}, "Dienstag": {}, "Mittwoch": {}, "Donnerstag": {}, "Freitag": {}, "Samstag": {}, "Sonntag": {}} 
    for tag in wochentage:
        wochentage[tag] = {"nachts_morgens": 0, "vormittags": 0, "mittags": 0, "nachmittags": 0, "abends_nachts": 0}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select dayofweek(committed_date_real) as dom, time(committed_date_real) as daytime
        from commits where year(committed_date_real)<2022 and month(committed_date_real) <12''')
    daytimes = []
    for day, daytime in mycursor:
        # daytimes.append(daytime)
        day_str = wochentage_lst[day-1]
        daytime_hours = daytime.total_seconds()/3600
        if daytime_hours >= 0 and daytime_hours < 6:
            slot = "nachts_morgens" 
        elif daytime_hours >= 6 and daytime_hours < 11:
            slot = "vormittags" 
        elif daytime_hours >= 11 and daytime_hours < 14:
            slot = "mittags" 
        elif daytime_hours >= 14 and daytime_hours < 19:
            slot = "nachmittags" 
        elif daytime_hours >= 19 and daytime_hours < 24:
            slot = "abends_nachts" 
        wochentage[day_str][slot] += 1
    mycursor.close()
    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,len(wochentage)-1,len(wochentage)))
    ax.set_xticklabels([day for day in wochentage])
    fig.set_size_inches(30,5)
    # plt.title("Verteilung der Commits auf Wochentage und Uhrzeit")

    plt.subplot(1,7,1)
    plt.xlabel("Montag", fontsize=25,fontweight='bold')
    patches, texts = plt.pie([value for value in wochentage["Montag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Montag"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,2)
    plt.xlabel("Dienstag", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Dienstag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Dienstag"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,3)
    plt.xlabel("Mittwoch", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Mittwoch"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Mittwoch"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,4)
    plt.xlabel("Donnerstag", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Donnerstag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Donnerstag"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,5)
    plt.xlabel("Freitag", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Freitag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Freitag"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,6)
    plt.xlabel("Samstag", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Samstag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Samstag"].values()]))/np.pi)**0.5)/250)

    plt.subplot(1,7,7)
    plt.xlabel("Sonntag", fontsize=25,fontweight='bold')
    plt.pie([value for value in wochentage["Sonntag"].values()], radius=((25569*(1+25569/sum([value for value in wochentage["Sonntag"].values()]))/np.pi)**0.5)/250)

    labels=["0 bis 6 Uhr","6 bis 11 Uhr","11 bis 14 Uhr","14 bis 19 Uhr","19 bis 24 Uhr", ]
    fig.legend(patches, labels, loc="upper center", ncol=5, fontsize=18)
    # handles, labels = plt.gca().get_legend_handles_labels()
    # by_label = dict(zip(labels, handles))
    # plt.legend(by_label.values(), by_label.keys())
    # plt.legend(loc='upper left', ncol=5)
    plt.savefig("FRAGE_7_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()
# frage7()

### Frage 8: Wann werden die Zeilen geändert?

In [ ]:
def frage8():
    jahre = {} 
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select sum(insertions) as ins, sum(deletions) as dels,  year(committed_date_real) as y, month(committed_date_real) as m from
        (select * from stats
        join commits
        where stats.commit_id = commits.id) as t
        group by y, m''')
    for ins, dels, jahr, monat in mycursor:
        if str(jahr) not in jahre.keys():
            jahre[str(jahr)] = {}
        jahre[str(jahr)][str(monat)] = {"ins": ins, "dels":dels}
    mycursor.close()

    fig, ax = plt.subplots()
    
    ax.ticklabel_format(useOffset=False, style='plain')
    # ax.set_xticks(np.linspace(0,15,16)+1)
    fig.set_size_inches(30,15)
    plt.xlabel("Jahr", fontsize=25,fontweight='bold')
    plt.ylabel("Anzahl Zeilenänderung", fontsize=25,fontweight='bold')
    # plt.title("Verteilung der Zeilenänderung über die Jahre")
    for jahr in jahre:
        # colors_green = iter([plt.cm.tab20c(i) for i in range(12)])
        # colors_red = iter([plt.cm.tab20c(i) for i in range(12)])
        cmap_g = mcp.gen_color(cmap="Greens",n=16)
        cmap_r = mcp.gen_color(cmap="Reds",n=16)
        bottom_bar_green = 0
        bottom_bar_red = 0
        for value in jahre[jahr]:
            plt.bar(int(jahr)-0.2, jahre[jahr][value]["ins"], bottom=bottom_bar_green, label=calendar.month_name[int(value)]+" (+)" , color=cmap_g[int(value)-1], width=0.35)
            # plt.bar(int(jahr)-0.2, jahre[jahr][value]["ins"], bottom=bottom_bar_green, label=calendar.month_name[int(value)] , color=[next(colors_green)], width=0.35)
            bottom_bar_green += jahre[jahr][value]["ins"]
            plt.bar(int(jahr)+0.2, jahre[jahr][value]["dels"], bottom=bottom_bar_red, label=calendar.month_name[int(value)]+" (-)" , color=cmap_r[int(value)-1], width=0.35)
            # plt.bar(int(jahr)+0.2, jahre[jahr][value]["dels"], bottom=bottom_bar_red, label=calendar.month_name[int(value)] , color=[next(colors_red)], width=0.35)
            bottom_bar_red += jahre[jahr][value]["dels"]
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    values = [val for val in by_label.values()]
    keys = [lab for lab in by_label.keys()]
    plt.legend(values[::-1], keys[::-1], fontsize=18)
    # plt.legend(loc='upper left', ncol=5)
    plt.savefig("FRAGE_8_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    wochentage_lst = ['Sonntag', 'Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag']
    wochentage = {"Montag": {}, "Dienstag": {}, "Mittwoch": {}, "Donnerstag": {}, "Freitag": {}, "Samstag": {}, "Sonntag": {}} 
    for tag in wochentage:
        wochentage[tag] = {"nachts_morgens": {"ins":0, "dels":0}, "vormittags": {"ins":0, "dels":0}, "mittags": {"ins":0, "dels":0}, "nachmittags": {"ins":0, "dels":0}, "abends_nachts": {"ins":0, "dels":0}}

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select ins, dels, dow, daytime from
        (select sum(insertions) as ins, sum(deletions) as dels, id, dayofweek(committed_date_real) as dow, time(committed_date_real) as daytime
        from
        (select * from stats
        join commits
        where stats.commit_id = commits.id) as t
        group by id) as a''')
    daytimes = []
    for ins, dels, day, daytime in mycursor:
        # daytimes.append(daytime)
        day_str = wochentage_lst[day-1]
        daytime_hours = daytime.total_seconds()/3600
        if daytime_hours >= 0 and daytime_hours < 6:
            slot = "nachts_morgens" 
        elif daytime_hours >= 6 and daytime_hours < 11:
            slot = "vormittags" 
        elif daytime_hours >= 11 and daytime_hours < 14:
            slot = "mittags" 
        elif daytime_hours >= 14 and daytime_hours < 19:
            slot = "nachmittags" 
        elif daytime_hours >= 19 and daytime_hours < 24:
            slot = "abends_nachts" 
        wochentage[day_str][slot]["ins"] += int(ins)
        wochentage[day_str][slot]["dels"] += dels
    mycursor.close()
    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,len(wochentage)-1,len(wochentage)))
    ax.set_xticklabels([day for day in wochentage])
    fig.set_size_inches(30,5)
    plt.title("Verteilung der Zeilenänderung auf Wochentage und Uhrzeit")
    
    cmap_g = mcp.gen_color(cmap="Greens",n=10)
    cmap_r = mcp.gen_color(cmap="Reds",n=10)
    colors_all = cmap_g[5:] + cmap_r[5:]
    plt.subplot(1,7,1)
    plt.xlabel("Montag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Montag"].values()] + [value["dels"] for value in wochentage["Montag"].values()]
    # print(values)
    # print(colors_all)
    patches, texts = plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)

    plt.subplot(1,7,2)
    plt.xlabel("Dienstag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Dienstag"].values()] + [value["dels"] for value in wochentage["Dienstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Dienstag"].values()])+sum([value["dels"] for value in wochentage["Dienstag"].values())/168206990)

    plt.subplot(1,7,3)
    plt.xlabel("Mittwoch", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Mittwoch"].values()] + [value["dels"] for value in wochentage["Mittwoch"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Mittwoch"].values()])+sum([value["dels"] for value in wochentage["Mittwoch"].values())/168206990)

    plt.subplot(1,7,4)
    plt.xlabel("Donnerstag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Donnerstag"].values()] + [value["dels"] for value in wochentage["Donnerstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Donnerstag"].values()])+sum([value["dels"] for value in wochentage["Donnerstag"].values())/168206990)

    plt.subplot(1,7,5)
    plt.xlabel("Freitag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Freitag"].values()] + [value["dels"] for value in wochentage["Freitag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Freitag"].values()])+sum([value["dels"] for value in wochentage["Freitag"].values())/168206990)

    plt.subplot(1,7,6)
    plt.xlabel("Samstag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Samstag"].values()] + [value["dels"] for value in wochentage["Samstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Samstag"].values()])+sum([value["dels"] for value in wochentage["Samstag"].values())/168206990)

    plt.subplot(1,7,7)
    plt.xlabel("Sonntag", labelpad=20, fontsize=25,fontweight='bold')
    values = [value["ins"] for value in wochentage["Sonntag"].values()] + [value["dels"] for value in wochentage["Sonntag"].values()]
    # values_used = [values[0],values[5], values[1], values[6], values[2], values[7], values[3], values[8], values[4], values[9]]
    # colors_used = [colors_all[0],colors_all[5], colors_all[1], colors_all[6], colors_all[2], colors_all[7], colors_all[3], colors_all[8], colors_all[4], colors_all[9]]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/168206990*7)#, radius=sum([value["ins"] for value in wochentage["Sonntag"].values()])+sum([value["dels"] for value in wochentage["Sonntag"].values()])/168206990)
    print(patches)
    patches_new = [patches[0], patches[5], patches[1], patches[6], patches[2], patches[7], patches[3], patches[8], patches[4], patches[9]]
    labels=["0 bis 6 Uhr (+)","0 bis 6 Uhr (-)" ,"6 bis 11 Uhr (+)","6 bis 11 Uhr (-)", "11 bis 14 Uhr (+)","11 bis 14 Uhr (-)","14 bis 19 Uhr (+)","14 bis 19 Uhr (-)","19 bis 24 Uhr (+)","19 bis 24 Uhr (-)" ]
    fig.legend(patches_new, labels, loc="upper center", ncol=5, fontsize=17)
    # handles, labels = plt.gca().get_legend_handles_labels()
    # by_label = dict(zip(labels, handles))
    # plt.legend(by_label.values(), by_label.keys())
    # plt.legend(loc='upper left', ncol=5)
    plt.savefig("FRAGE_8_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

# frage8()

### Frage 9: Wann werden die Fehler committet?

In [ ]:
def frage9():
    jahre = {} 
    sum_ff = 0
    sum_cp = 0
    sum_cp_er = 0
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id) as count, year(committed_date_real) as y, month(committed_date_real) as m
        from flawfinder
        join commits
        where commits.id = flawfinder.commit_id
        group by y, m''')
    for count, jahr, monat in mycursor:
        if str(jahr) not in jahre.keys():
            jahre[str(jahr)] = {}
        jahre[str(jahr)][str(monat)] = {"flawfinder": count, "cppcheck":0, "cppcheck_error":0}
        sum_ff += count
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id) as count, year(committed_date_real) as y, month(committed_date_real) as m
        from cppcheck
        join commits
        where commits.id = cppcheck.commit_id
        group by y, m''')
    for count, jahr, monat in mycursor:
        jahre[str(jahr)][str(monat)]["cppcheck"] = count
        sum_cp += count
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id) as count, year(committed_date_real) as y, month(committed_date_real) as m
        from cppcheck
        join commits
        where commits.id = cppcheck.commit_id and categorie="error"
        group by y, m''')
    for count, jahr, monat in mycursor:
        jahre[str(jahr)][str(monat)]["cppcheck_error"] = count
        sum_cp_er += count
    mycursor.close()



    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    # ax.set_xticks(np.linspace(0,15,16)+1)
    fig.set_size_inches(30,15)
    plt.xlabel("Jahr", fontsize=25,fontweight='bold')
    plt.ylabel("Anzahl Fehler \n (relativer Anteil bezogen auf die Gesamtanzahl)", fontsize=25,fontweight='bold')
    # plt.title("Verteilung der relativen Anzahl Fehler über die Jahre (Flawfinder, Cppcheck, Errormeldungen von Cppcheck)")
    for jahr in jahre:
        # colors_green = iter([plt.cm.tab20c(i) for i in range(12)])
        # colors_red = iter([plt.cm.tab20c(i) for i in range(12)])
        cmap_g = mcp.gen_color(cmap="winter",n=16)
        cmap_r = mcp.gen_color(cmap="autumn",n=16)
        cmap_o = mcp.gen_color(cmap="autumn",n=16)
        bottom_bar_green = 0
        bottom_bar_red = 0
        bottom_bar_orange = 0
        for value in jahre[jahr]:
            plt.bar(int(jahr)-0.2, jahre[jahr][value]["flawfinder"]/sum_ff, bottom=bottom_bar_green, label=calendar.month_name[int(value)], color=cmap_g[int(value)-1], width=0.18)
            bottom_bar_green += jahre[jahr][value]["flawfinder"]/sum_ff
            plt.bar(int(jahr), jahre[jahr][value]["cppcheck"]/sum_cp, bottom=bottom_bar_orange, label=calendar.month_name[int(value)]+" " , color=cmap_r[int(value)-1], width=0.18)
            bottom_bar_orange += jahre[jahr][value]["cppcheck"]/sum_cp
            plt.bar(int(jahr)+0.2, jahre[jahr][value]["cppcheck_error"]/sum_cp_er, bottom=bottom_bar_red, color=cmap_o[int(value)-1], width=0.18)
            bottom_bar_red += jahre[jahr][value]["cppcheck_error"]/sum_cp_er
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    order = [0,2,4,6,8,10,12,14,16,18,20,22, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]
    legend1 = [handles[idx] for idx in order]
    legend2 = [labels[idx] for idx in order]
    legend = plt.legend(legend1[::-1], legend2[::-1], title = " Cppcheck         Flawfinder", ncol=2, fontsize=18)
    plt.setp(legend.get_title(),fontsize=23)
    # plt.legend(loc='upper left')
    plt.savefig("FRAGE_9_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    wochentage_lst = ['Sonntag', 'Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag']
    wochentage = {"Montag": {}, "Dienstag": {}, "Mittwoch": {}, "Donnerstag": {}, "Freitag": {}, "Samstag": {}, "Sonntag": {}} 
    for tag in wochentage:
        wochentage[tag] = {"nachts_morgens": {"cppcheck":0, "flawfinder":0}, "vormittags": {"cppcheck":0, "flawfinder":0}, "mittags": {"cppcheck":0, "flawfinder":0}, "nachmittags": {"cppcheck":0, "flawfinder":0}, "abends_nachts": {"cppcheck":0, "flawfinder":0}}

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id) as count, dayofweek(committed_date_real) as dow, time(committed_date_real) as daytime
        from cppcheck
        join commits
        where commits.id = cppcheck.commit_id and cppcheck.categorie="error"
        group by commit_id''')
    daytimes = []
    for count, day, daytime in mycursor:
        # daytimes.append(daytime)
        day_str = wochentage_lst[day-1]
        daytime_hours = daytime.total_seconds()/3600
        if daytime_hours >= 0 and daytime_hours < 6:
            slot = "nachts_morgens" 
        elif daytime_hours >= 6 and daytime_hours < 11:
            slot = "vormittags" 
        elif daytime_hours >= 11 and daytime_hours < 14:
            slot = "mittags" 
        elif daytime_hours >= 14 and daytime_hours < 19:
            slot = "nachmittags" 
        elif daytime_hours >= 19 and daytime_hours < 24:
            slot = "abends_nachts" 
        wochentage[day_str][slot]["cppcheck"] += count
    mycursor.close()
    
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id) as count, dayofweek(committed_date_real) as dow, time(committed_date_real) as daytime
        from flawfinder
        join commits
        where commits.id = flawfinder.commit_id
        group by commit_id''')
    daytimes = []
    for count, day, daytime in mycursor:
        # daytimes.append(daytime)
        day_str = wochentage_lst[day-1]
        daytime_hours = daytime.total_seconds()/3600
        if daytime_hours >= 0 and daytime_hours < 6:
            slot = "nachts_morgens" 
        elif daytime_hours >= 6 and daytime_hours < 11:
            slot = "vormittags" 
        elif daytime_hours >= 11 and daytime_hours < 14:
            slot = "mittags" 
        elif daytime_hours >= 14 and daytime_hours < 19:
            slot = "nachmittags" 
        elif daytime_hours >= 19 and daytime_hours < 24:
            slot = "abends_nachts" 
        wochentage[day_str][slot]["flawfinder"] += count
    mycursor.close()
    fig, ax = plt.subplots()
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.set_xticks(np.linspace(0,len(wochentage)-1,len(wochentage)))
    ax.set_xticklabels([day for day in wochentage])
    ax.text(10, 2,"CppCheck \n Flawfinder", fontsize=20)
    fig.set_size_inches(30,5)
    plt.title("Verteilung der Anzahl Fehler auf Wochentage und Uhrzeit")
    
    cmap_g = mcp.gen_color(cmap="autumn",n=10)
    cmap_r = mcp.gen_color(cmap="winter",n=10)
    colors_all = cmap_g[5:] + cmap_r[5:]
    plt.subplot(1,7,1)
    plt.xlabel("Montag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Montag"].values()]+ [value["flawfinder"] for value in wochentage["Montag"].values()]
    # print(values)
    # print(colors_all)
    patches, texts = plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)

    plt.subplot(1,7,2)
    plt.xlabel("Dienstag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Dienstag"].values()] + [value["flawfinder"] for value in wochentage["Dienstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Dienstag"].values()])+sum([value["dels"] for value in wochentage["Dienstag"].values())/168206990)

    plt.subplot(1,7,3)
    plt.xlabel("Mittwoch", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Mittwoch"].values()]+ [value["flawfinder"] for value in wochentage["Mittwoch"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Mittwoch"].values()])+sum([value["dels"] for value in wochentage["Mittwoch"].values())/168206990)

    plt.subplot(1,7,4)
    plt.xlabel("Donnerstag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Donnerstag"].values()] + [value["flawfinder"] for value in wochentage["Donnerstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Donnerstag"].values()])+sum([value["dels"] for value in wochentage["Donnerstag"].values())/168206990)

    plt.subplot(1,7,5)
    plt.xlabel("Freitag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Freitag"].values()] + [value["flawfinder"] for value in wochentage["Freitag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Freitag"].values()])+sum([value["dels"] for value in wochentage["Freitag"].values())/168206990)

    plt.subplot(1,7,6)
    plt.xlabel("Samstag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Samstag"].values()] + [value["flawfinder"] for value in wochentage["Samstag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Samstag"].values()])+sum([value["dels"] for value in wochentage["Samstag"].values())/168206990)

    plt.subplot(1,7,7)
    plt.xlabel("Sonntag", fontsize=25,fontweight='bold')
    values = [value["cppcheck"] for value in wochentage["Sonntag"].values()] + [value["flawfinder"] for value in wochentage["Sonntag"].values()]
    plt.pie(values, colors=colors_all, radius=sum(np.array(values, dtype=float))/14431)#, radius=sum([value["ins"] for value in wochentage["Sonntag"].values()])+sum([value["dels"] for value in wochentage["Sonntag"].values()])/168206990)
    # print(patches)
    patches_new = [patches[0], patches[5], patches[1], patches[6], patches[2], patches[7], patches[3], patches[8], patches[4], patches[9]]
    labels=["0 bis 6 Uhr (+)","0 bis 6 Uhr (-)" ,"6 bis 11 Uhr (+)","6 bis 11 Uhr (-)", "11 bis 14 Uhr (+)","11 bis 14 Uhr (-)","14 bis 19 Uhr (+)","14 bis 19 Uhr (-)","19 bis 24 Uhr (+)","19 bis 24 Uhr (-)" ]

    legend = fig.legend(patches_new, labels, loc="upper center", ncol=5, fontsize=18)
    
    # handles, labels = plt.gca().get_legend_handles_labels()
    # by_label = dict(zip(labels, handles))
    # plt.legend(by_label.values(), by_label.keys())
    # plt.legend(loc='upper left', ncol=5)
    plt.savefig("FRAGE_9_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()
# frage9()

### Frage 10: Welche Dateien werden gemeinsam verändert (Couplings)?


In [ ]:
def frage10():
    coups, colors1, colors2, back, categorie_coups = getcolorsCouplings()
    print(tabulate(coups, headers=["Pfad1", "Pfad2", "Anzahl Couplings"], tablefmt='latex'))  

def print_with_color(s, color_fore=Fore.WHITE, color_back=Back.BLUE, brightness=Style.NORMAL, **kwargs):
    print(f"{brightness}{color_fore+color_back}{s}{Style.RESET_ALL}", **kwargs)

def getcolorsCouplings():
    categories = ["Quellcode", "Grafik", "Konfiguration", "Text", "Sonstiges", "Versionskontrolle", "Repo_Konfig", "Sprache"]
    colors_fore = [Fore.CYAN, Fore.BLUE, Fore.RED, Fore.YELLOW, Fore.WHITE, Fore.MAGENTA, Fore.LIGHTRED_EX, Fore.BLACK]
    colors_back = [Back.LIGHTGREEN_EX, Back.LIGHTRED_EX]
    colors_html_fore = ["#B22222", "#FF1493", "#6A5ACD", "#00FF00", "#006400", "#DAA520", "#00FFFF", "#C0C0C0"]
    # preperation lists
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(c.id)
        FROM files as f
        right join couplings as c 
        on f.file_name=c.file_name1
        where categorie is not null''')
    count_couplings1 = mycursor.fetchall()[0][0]
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select max(c.id)
        FROM files as f
        right join couplings as c 
        on f.file_name=c.file_name2
        where categorie is not null''')
    count_couplings2 = mycursor.fetchall()[0][0]
    mycursor.close()
    if count_couplings1 > count_couplings2:
        data = [[]] * count_couplings1
    else:
        data = [[]] * count_couplings2

    # data in lists
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select c.id, file_name1, file_name2, categorie, count
        FROM files as f
        right join couplings as c 
        on f.file_name=c.file_name1
        where categorie is not null''')
    for coupling_id, file1, file2, categorie1, count in mycursor:
        data[coupling_id-1] = [count, file1, categorie1, file2]  
    mycursor.close()
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select c.id, file_name2, categorie
        FROM files as f
        right join couplings as c 
        on f.file_name=c.file_name2
        where categorie is not null''')
    for coupling_id, file2, categorie2 in mycursor:
        data[coupling_id-1].append(categorie2)
    mycursor.close()
    # print(data[:6])
    #generate result
    color1 = []
    color2 = []
    coups = []
    categorie_coups = []
    for i in range(len(data)):
        if len(data[i])==5:
            count = data[i][0]
            if count > 1000:
                categorie1 = data[i][2]
                categorie2 = data[i][4]
                categorie_coups.append([categorie1, categorie2, count])
                color1.append(colors_html_fore[categories.index(categorie1)])
                color2.append(colors_html_fore[categories.index(categorie2)])
                if categorie1 == categorie2:
                    back = colors_back[0]
                else:
                    back = colors_back[1]
                coups.append([f"{data[i][1]}", f"{data[i][3]}", f"{count}"])
    # print(coups[:5])
    return coups, color1, color2, back, categorie_coups

def saveHTMLCouplings():
    categories = ["Quellcode", "Grafik", "Konfiguration", "Text", "Sonstiges", "Versionskontrolle", "Repo_Konfig", "Sprache"]
    colors_fore = [Fore.CYAN, Fore.BLUE, Fore.RED, Fore.YELLOW, Fore.WHITE, Fore.MAGENTA, Fore.LIGHTRED_EX, Fore.BLACK]
    colors_back = [Back.LIGHTGREEN_EX, Back.LIGHTRED_EX]
    colors_html_fore = ["#B22222", "#FF1493", "#6A5ACD", "#00FF00", "#006400", "#DAA520", "#00FFFF", "#C0C0C0"]
    coups, colors1, colors2, back, categorie_coups = getcolorsCouplings()
    html_str = tabulate(coups, headers=["Pfad1", "Pfad2", "Anzahl Couplings"], tablefmt='html')
    html_str_new = ""
    i=0
    for line in html_str.splitlines():
        result = line.find("<td>")
        if result != -1:
            begin_str1 = line[:result+3]
            end_str1 = line[result+3:]
            result = end_str1.find("<td>")
            begin_str2 = end_str1[:result+3]
            end_str_2 = end_str1[result+3:]
            # result = end_str_2.find("<td>")
            # begin_str3 = end_str_2[:result+3]
            # end_str_3 = end_str_2[result+3:]
            # line_new = begin_str1+ ' style="background-color:'+ colors1[i]+'"'+ begin_str2+' style="background-color:'+ colors2[i]+'"'+ begin_str3 +' style="background-color:'+ back[i]+'"'+ end_str_3 +"\n"
            # print(line_new)
            line_new = begin_str1+ ' style="background-color:'+ colors1[i]+'"'+ begin_str2+' style="background-color:'+ colors2[i]+'"'+ end_str_2 + "\n"
            i += 1
        else:
            line_new = line + "\n"
        html_str_new += line_new
    legende = '<!DOCTYPE html> <html> <head> <style> div {   background-color: lightgrey;   width: 780px;   padding: 30px; } </style> </head> <body> <div>  <p> <span style="background-color:'+colors_html_fore[0]+'">'+categories[0]+'</span> &emsp; <span style="background-color:'+colors_html_fore[1]+'">'+categories[1]+'</span> &emsp; <span style="background-color:'+colors_html_fore[2]+'">'+categories[2]+'</span> &emsp; <span style="background-color:'+colors_html_fore[3]+'">'+categories[3]+'</span> &emsp; <span style="background-color:'+colors_html_fore[4]+'">'+categories[4]+'</span> &emsp; <span style="background-color:'+colors_html_fore[5]+'">'+categories[5]+'</span> &emsp; <span style="background-color:'+colors_html_fore[6]+'">'+categories[6]+'</span> &emsp; <span style="background-color:'+colors_html_fore[7]+'">'+categories[7]+'</span> </p> </div> '
    ende = '</body> </html>'
    Html_file= open("E:\Bachelor\Gitlab_Inkscape\\visualisierung\\coloredCouplings_sql.html","w")
    Html_file.write(legende+html_str_new+ende)
    Html_file.close()
    # print(html_str_new)

def frage10_html_old_colored():
    categories = ["Quellcode", "Grafik", "Konfiguration", "Text", "Sonstiges", "Versionskontrolle", "Repo_Konfig", "Sprache"]
    colors_fore = [Fore.CYAN, Fore.BLUE, Fore.RED, Fore.YELLOW, Fore.WHITE, Fore.MAGENTA, Fore.LIGHTRED_EX, Fore.BLACK]
    colors_back = [Back.LIGHTGREEN_EX, Back.LIGHTRED_EX]
    colors_html_fore = ["#B22222", "#FF1493", "#6A5ACD", "#00FF00", "#006400", "#DAA520", "#00FFFF", "#C0C0C0"]
    saveHTMLCouplings()

def frage10_endungen():
    
    endings = {}
    coups, colors1, colors2, back, categorie_coups = getcolorsCouplings()
    colors = {".cpp": "blue", ".po": "green", ".in":"lime", ".pot":"olive", ".h":"royalblue", ".ac":"yellow", "0":"gray", ".mingw":"orange", ".svg":"magenta"}
    for coup in coups:
        if str(getEnding(coup[0]))+" & "+str(getEnding(coup[1])) in endings:
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))]["count"] += int(coup[2])
        else:
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))] = {}
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))]["name"] = str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))]["count"]  = int(coup[2])
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))]["color_left"] = colors[str(getEnding(coup[0]))]
            endings[str(getEnding(coup[0]))+" & "+str(getEnding(coup[1]))]["color_right"] = colors[str(getEnding(coup[1]))]
    endings_new = sorted(endings.values(), key=itemgetter("count"))
    x_ticks=[]
    x=[]
    y=[]
    color_left=[]
    color_right=[]
    for i in range(len(endings_new)):
        x.append(i)
        x_ticks.append(endings_new[i]["name"])
        y.append(endings_new[i]["count"])
        color_left.append(endings_new[i]["color_left"])
        color_right.append(endings_new[i]["color_right"])
    fig, ax = plt.subplots(dpi=240) 
    fig.set_size_inches(30,15)
    plt.xticks(x, x_ticks,  fontsize=20)
    plt.bar([x_left-0.2 for x_left in x], y, color=color_left, align='center', width=0.4)
    plt.bar([x_rechts+0.2 for x_rechts in x], y, color=color_right, align='center', width=0.4)
    plt.ylabel("Anzahl Couplings", fontsize=30,fontweight='bold')
    plt.xlabel("Dateiendungen", fontsize=30,fontweight='bold')
    plt.savefig("FRAGE_10.png",dpi=240, bbox_inches = 'tight')
    plt.show()

# frage10_endungen()
# frage10_html_old_colored()
# frage10()

### Frage 11: Wie lassen sich Commits klassifizieren?

In [ ]:
def frage11_bild():
    ins_data = []
    dels_data = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select id, ins, dels from
        (select commits.id as id, author, authoredDate, committer, committed_Date,sum(insertions) as ins, sum(deletions) as dels from commits
        join stats
        where stats.commit_id=commits.id
        group by commits.id) as a''')
    for c_id, ins, dels in mycursor:
        if ins == 0 and dels == 0:
            ins_data.append(0)
            dels_data.append(0)
        else:
            ins_data.append(np.log10(ins+1))
            dels_data.append(np.log10(dels+1))

    mycursor.close()



    ## 1: gleiche viele verändert
    ## 2: nur hinzugefügt
    ## 3: nur gelöscht
    ## 4: mehr hinzugefügt als gelöscht
    ## 5 mehr gelöscht als hinzugefügt

    klassen = {"1":{"x": [], "y": []}, "2":{"x": [], "y": []}, "3":{"x": [], "y": []}, "4":{"x": [], "y": []}, "5":{"x": [], "y": []}}

    for ins, dels in zip(ins_data, dels_data):
        if ins == dels:
            klassen["1"]["x"].append(ins)
            klassen["1"]["y"].append(dels)
        elif ins != 0 and dels == 0:
            klassen["2"]["x"].append(ins)
            klassen["2"]["y"].append(dels)
        elif ins == 0 and dels != 0:
            klassen["3"]["x"].append(ins)
            klassen["3"]["y"].append(dels)
        elif ins > dels:
            klassen["4"]["x"].append(ins)
            klassen["4"]["y"].append(dels)
        elif ins < dels:
            klassen["5"]["x"].append(ins)
            klassen["5"]["y"].append(dels)



    fig, ax = plt.subplots() 
    fig.set_size_inches(25, 15)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xticks([0, np.log10(2), np.log10(6), np.log10(11), np.log10(101), np.log10(1001), np.log10(10001), np.log10(100001), np.log10(1000001), np.log10(3000001)],[0, 1,5, 10, 100, 1000, 10000, 100000, 1000000, 3000000])
    plt.yticks([0, np.log10(2), np.log10(6), np.log10(11), np.log10(101), np.log10(1001), np.log10(10001), np.log10(100001), np.log10(1000001), np.log10(3000001)],[0, 1,5, 10, 100, 1000, 10000, 100000, 1000000, 3000000])
    plt.xlabel("Hinzugefügte Zeilen", fontsize=35,fontweight='bold')
    plt.ylabel("Gelöschte Zeilen", fontsize=35,fontweight='bold')
    plt.scatter(klassen["1"]["x"], klassen["1"]["y"], alpha=0.1, color="blue")
    plt.scatter(klassen["2"]["x"], klassen["2"]["y"], alpha=0.1, color="yellow")
    plt.scatter(klassen["3"]["x"], klassen["3"]["y"], alpha=0.1, color="cyan")
    plt.scatter(klassen["4"]["x"], klassen["4"]["y"], alpha=0.1, color="green")
    plt.scatter(klassen["5"]["x"], klassen["5"]["y"], alpha=0.1, color="red")
    # plt.scatter(ins_data, dels_data, alpha=0.1)
    plt.savefig("FRAGE_11.png",dpi=240, bbox_inches = 'tight')
    plt.show()

def frage11_rechnung():
    ins_data = []
    dels_data = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select id, ins, dels from
        (select commits.id as id, author, authoredDate, committer, committed_Date,sum(insertions) as ins, sum(deletions) as dels from commits
        join stats
        where stats.commit_id=commits.id
        group by commits.id) as a''')
    for c_id, ins, dels in mycursor:
        ins_data.append(ins)
        dels_data.append(dels)

    mycursor.close()

    klassen = {"1":{"x": [], "y": []}, "2":{"x": [], "y": []}, "3":{"x": [], "y": []}, "4":{"x": [], "y": []}, "5":{"x": [], "y": []}}

    for ins, dels in zip(ins_data, dels_data):
        if ins == dels:
            klassen["1"]["x"].append(ins)
            klassen["1"]["y"].append(dels)
        elif ins != 0 and dels == 0:
            klassen["2"]["x"].append(ins)
            klassen["2"]["y"].append(dels)
        elif ins == 0 and dels != 0:
            klassen["3"]["x"].append(ins)
            klassen["3"]["y"].append(dels)
        elif ins > dels:
            klassen["4"]["x"].append(ins)
            klassen["4"]["y"].append(dels)
        elif ins < dels:
            klassen["5"]["x"].append(ins)
            klassen["5"]["y"].append(dels)

    punkte_1 = [np.array((x, y)) for x, y in zip(klassen["1"]["x"], klassen["1"]["y"])]
    punkte_2 = [np.array((x, y)) for x, y in zip(klassen["2"]["x"], klassen["2"]["y"])]
    punkte_3 = [np.array((x, y)) for x, y in zip(klassen["3"]["x"], klassen["3"]["y"])]
    punkte_4 = [np.array((x, y)) for x, y in zip(klassen["4"]["x"], klassen["4"]["y"])]
    punkte_5 = [np.array((x, y)) for x, y in zip(klassen["5"]["x"], klassen["5"]["y"])]

    combis_1 = list(combinations(punkte_1, 2))
    combis_2 = list(combinations(punkte_2, 2))
    combis_3 = list(combinations(punkte_3, 2))
    combis_4 = list(combinations(punkte_4, 2))
    combis_5 = list(combinations(punkte_5, 2))

    dist_1 = []
    dist_2 = []
    dist_3 = []
    dist_4 = []
    dist_5 = []

    dist_1 = []
    for combi in combis_1:
        dist_1.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    dist_2 = []
    for combi in combis_2:
        dist_2.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    dist_3 = []
    for combi in combis_3:
        dist_3.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    dist_4 = []
    for combi in combis_4:
        dist_4.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    dist_5 = []
    for combi in combis_5:
        dist_5.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))

    print("hSUM(K1) = ",sum(dist_1),"mit c = 1\nhSCHNITT(K1) = ", sum(dist_1)/(0.5*len(combis_1)*(len(combis_1)-1)), "cSchnitt = ", (0.5*len(combis_1)*(len(combis_1)-1)))
    print("hSUM(K2) = ",sum(dist_2),"mit c = 1\nhSCHNITT(K1) = ", sum(dist_2)/(0.5*len(combis_2)*(len(combis_2)-1)), "cSchnitt = ", (0.5*len(combis_1)*(len(combis_2)-1)))
    print("hSUM(K3) = ",sum(dist_3),"mit c = 1\nhSCHNITT(K1) = ", sum(dist_3)/(0.5*len(combis_3)*(len(combis_3)-1)), "cSchnitt = ", (0.5*len(combis_1)*(len(combis_3)-1)))
    print("hSUM(K4) = ",sum(dist_4),"mit c = 1\nhSCHNITT(K1) = ", sum(dist_4)/(0.5*len(combis_4)*(len(combis_4)-1)), "cSchnitt = ", (0.5*len(combis_1)*(len(combis_4)-1)))
    print("hSUM(K5) = ",sum(dist_5),"mit c = 1\nhSCHNITT(K1) = ", sum(dist_5)/(0.5*len(combis_5)*(len(combis_5)-1)), "cSchnitt = ", (0.5*len(combis_1)*(len(combis_5)-1)))
    global h_schnitt_cppcheck_summe
    global h_summe_cppcheck_summe
    h_summe_cppcheck_summe = sum(dist_1)+sum(dist_2)+sum(dist_3)+sum(dist_4)+sum(dist_5)
    h_schnitt_cppcheck_summe = sum(dist_1)/(0.5*len(combis_1)*(len(combis_1)-1))+sum(dist_2)/(0.5*len(combis_2)*(len(combis_2)-1))+sum(dist_3)/(0.5*len(combis_3)*(len(combis_3)-1))+sum(dist_4)/(0.5*len(combis_4)*(len(combis_4)-1))+sum(dist_5)/(0.5*len(combis_5)*(len(combis_5)-1))
# frage11_bild()
# frage11_rechnung()


In [ ]:
# only for inkscape project
# print("---------Bewertung Commit-----------------------")
# print("Kombination aus h(K)Schnitt,Cppcheck und vSingle,Cppcheck(K1, K2): ", h_schnitt_cppcheck_summe, "/", sum([1,1,1,1,1.4142,1,2,2,1,1.4142]), "=", h_schnitt_cppcheck_summe/sum([1,1,1,1,1.4142,1,2,2,1,1.4142]))
# print("Kombination aus hSumme,Cppcheck(K) und vComplete,Cppcheck(K1, K2): ", h_summe_cppcheck_summe, "/",sum([2381445, 2381445, 2560940.8, 3691783.72, 3367871.8, 2560940.05, 3691783.1, 2564934.84, 3691782.94, 3691781.69]), "=", h_summe_cppcheck_summe/sum([2381445, 2381445, 2560940.8, 3691783.72, 3367871.8, 2560940.05, 3691783.1, 2564934.84, 3691782.94, 3691781.69]))


In [ ]:
def frage11_single_complete():
    ins_data = []
    dels_data = []
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    punkte_1_cpp = []
    punkte_2_cpp = []
    punkte_3_cpp = []
    punkte_4_cpp = []
    punkte_5_cpp = []
    mycursor.execute('''select id, ins, dels from
        (select commits.id as id, sum(insertions) as ins, sum(deletions) as dels from commits
        join stats
        where stats.commit_id=commits.id
        group by commits.id) as a''')
    for c_id, ins, dels in mycursor:
        if ins == dels:
            punkte_1_cpp.append([int(ins),int(dels)])
        elif ins != 0 and dels == 0:
            punkte_2_cpp.append([int(ins),int(dels)])
        elif ins == 0 and dels != 0:
            punkte_3_cpp.append([int(ins),int(dels)])
        elif ins > dels:
            punkte_4_cpp.append([int(ins),int(dels)])
        elif ins < dels:
            punkte_5_cpp.append([int(ins),int(dels)])
    mycursor.close()
    print("1. Datenerhebung fertig")    
    print("2. Einteilung Kassen fertig")
    
    print("3. Punkteextraktion fertig")
    punkte_1_2_cpp = [punkte_1_cpp, punkte_2_cpp] 
    punkte_1_3_cpp = [punkte_1_cpp, punkte_3_cpp] 
    punkte_1_4_cpp = [punkte_1_cpp, punkte_4_cpp] 
    punkte_1_5_cpp = [punkte_1_cpp, punkte_5_cpp] 
    punkte_2_3_cpp = [punkte_2_cpp, punkte_3_cpp] 
    punkte_2_4_cpp = [punkte_2_cpp, punkte_4_cpp] 
    punkte_2_5_cpp = [punkte_2_cpp, punkte_5_cpp] 
    punkte_3_4_cpp = [punkte_3_cpp, punkte_4_cpp] 
    punkte_3_5_cpp = [punkte_3_cpp, punkte_5_cpp] 
    punkte_4_5_cpp = [punkte_4_cpp, punkte_5_cpp] 
    
    print("4. Kombination Punkte fertig")
    combis_1_2_cpp = list(itertools.product(*punkte_1_2_cpp))
    combis_1_3_cpp = list(itertools.product(*punkte_1_3_cpp))
    combis_1_4_cpp = list(itertools.product(*punkte_1_4_cpp))
    combis_1_5_cpp = list(itertools.product(*punkte_1_5_cpp))
    combis_2_3_cpp = list(itertools.product(*punkte_2_3_cpp))
    combis_2_4_cpp = list(itertools.product(*punkte_2_4_cpp))
    combis_2_5_cpp = list(itertools.product(*punkte_2_5_cpp))
    combis_3_4_cpp = list(itertools.product(*punkte_3_4_cpp))
    combis_3_5_cpp = list(itertools.product(*punkte_3_5_cpp))
    combis_4_5_cpp = list(itertools.product(*punkte_4_5_cpp))
    print(combis_1_2_cpp[:3])
    print("5. Alle Kombis fertig")
    dist_1_2_cpp = []
    dist_1_3_cpp = []
    dist_1_4_cpp = []
    dist_1_5_cpp = []
    dist_2_3_cpp = []
    dist_2_4_cpp = []
    dist_2_5_cpp = []
    dist_3_4_cpp = []
    dist_3_5_cpp = []
    dist_4_5_cpp = []

    for combi in combis_1_2_cpp:
        dist_1_2_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist1 fertig")
    for combi in combis_1_3_cpp:
        dist_1_3_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist2 fertig")
    for combi in combis_1_4_cpp:
        dist_1_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist3 fertig")
    for combi in combis_1_5_cpp:
        dist_1_5_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist4 fertig")
    for combi in combis_2_3_cpp:
        dist_2_3_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist5 fertig")
    for combi in combis_2_4_cpp:
        dist_2_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist6 fertig")
    for combi in combis_2_5_cpp:
        dist_2_5_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist7 fertig")
    for combi in combis_3_4_cpp:
        dist_3_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist8 fertig")
    for combi in combis_3_5_cpp:
        dist_3_5_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist9 fertig")
    for combi in combis_4_5_cpp:
        dist_4_5_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("6. Dist10 fertig")

    
    print("-------------------CPPCHECK-------------------------------")
    print("v_single_cpp(K_1,K_2) = ",min(dist_1_2_cpp))
    print("v_complete_cpp(K_1,K_2) = ",max(dist_1_2_cpp))
    print("v_single_cpp(K_1,K_3) = ",min(dist_1_3_cpp))
    print("v_complete_cpp(K_1,K_3) = ",max(dist_1_3_cpp))
    print("v_single_cpp(K_1,K_4) = ",min(dist_1_4_cpp))
    print("v_complete_cpp(K_1,K_4) = ",max(dist_1_4_cpp))
    print("v_single_cpp(K_1,K_5) = ",min(dist_1_5_cpp))
    print("v_complete_cpp(K_1,K_5) = ",max(dist_1_5_cpp))
    print("v_single_cpp(K_2,K_3) = ",min(dist_2_3_cpp))
    print("v_complete_cpp(K_2,K_3) = ",max(dist_2_3_cpp))
    print("v_single_cpp(K_2,K_4) = ",min(dist_2_4_cpp))
    print("v_complete_cpp(K_2,K_4) = ",max(dist_2_4_cpp))
    print("v_single_cpp(K_2,K_5) = ",min(dist_2_5_cpp))
    print("v_complete_cpp(K_2,K_5) = ",max(dist_2_5_cpp))
    print("v_single_cpp(K_3,K_4) = ",min(dist_3_4_cpp))
    print("v_complete_cpp(K_3,K_4) = ",max(dist_3_4_cpp))
    print("v_single_cpp(K_3,K_5) = ",min(dist_3_5_cpp))
    print("v_complete_cpp(K_3,K_5) = ",max(dist_3_5_cpp))
    print("v_single_cpp(K_4,K_5) = ",min(dist_4_5_cpp))
    print("v_complete_cpp(K_4,K_5) = ",max(dist_4_5_cpp))


    print("----------------------LATEX Commit SINGLE--------------------------------")
    print("$v_{Single,Cppcheck}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_2_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_3_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_4_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_3_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_4_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_3_4_cpp),"$","\\","\\")

    print("----------------------LATEX Commit COMPLETE--------------------------------")
    print("$v_{Complete,Cppcheck}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_2_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_3_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_4_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_3_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_4_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_3_4_cpp),"$","\\","\\")

    print("---------Bewertung Commit-----------------------")
    print("Kombination aus h(K)Schnitt,Cppcheck und vSingle,Cppcheck(K1, K2): ", h_schnitt_cppcheck_summe, "/", sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]), "=", h_schnitt_cppcheck_summe/sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]))
    print("Kombination aus hSumme,Cppcheck(K) und vComplete,Cppcheck(K1, K2): ", h_summe_cppcheck_summe, "/",sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]), "=", h_summe_cppcheck_summe/sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]))

# frage11_single_complete()

### Frage 12: Wie lassen sich Autoren klassifizieren?

In [ ]:
def frage12():
    autoren = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(id) from commits
    group by author''')
    for a, count in mycursor:
        autoren[str(a)] = {"count_commits": count, "count_cpp": 0, "count_ff":0}
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(c_fehler) from
    (select  commits.author as author , cppcheck.fehler as c_fehler, categorie
    from cppcheck
    join commits
    where commits.id = cppcheck.commit_id) as a
    where categorie = "error"
    group by author''')
    for a, count in mycursor:
        autoren[str(a)]["count_cpp"] = count
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(c_fehler) from
    (select  commits.author as author , flawfinder.name as c_fehler, Level
    from flawfinder
    join commits
    where commits.id = flawfinder.commit_id) as a
    where Level > 2
    group by author''')
    for a, count in mycursor:
        autoren[str(a)]["count_ff"] = count
    mycursor.close()

    x_plt = [autoren[str(x)]["count_commits"] for x in autoren]
    y_plt_cpp = []
    for x in autoren:
        if autoren[str(x)]["count_cpp"] == 0:
            y_plt_cpp.append(-1)
        else:
            y_plt_cpp.append(autoren[str(x)]["count_cpp"])
    # y_plt_cpp = [autoren[str(x)]["count_cpp"] for x in autoren]
    # y_plt_ff = [autoren[str(x)]["count_ff"] for x in autoren]
    y_plt_ff = []
    for x in autoren:
        if autoren[str(x)]["count_ff"] == 0:
            y_plt_ff.append(-1)
        else:
            y_plt_ff.append(autoren[str(x)]["count_ff"])

    klassen_cpp = {"1": {"x":[], "y":[]}, "2": {"x":[], "y":[]}, "3": {"x":[], "y":[]}, "4": {"x":[], "y":[]}}
    klassen_ff = {"1": {"x":[], "y":[]}, "2": {"x":[], "y":[]}, "3": {"x":[], "y":[]}, "4": {"x":[], "y":[]}}
    # for c_count, cpp  _count in y_plt_cpp:
    for item in autoren:
        c_count = autoren[str(item)]["count_commits"]
        cpp_count = autoren[str(item)]["count_cpp"]
        ff_count = autoren[str(item)]["count_ff"]
        #cppcheck
        if cpp_count == 0:        
            klassen_cpp["1"]["x"].append(np.log10(c_count))
            klassen_cpp["1"]["y"].append(-1)
        elif cpp_count == c_count:
            klassen_cpp["2"]["x"].append(np.log10(c_count))
            klassen_cpp["2"]["y"].append(np.log10(cpp_count))
        elif cpp_count < c_count:
            klassen_cpp["3"]["x"].append(np.log10(c_count))
            klassen_cpp["3"]["y"].append(np.log10(cpp_count))
        elif cpp_count > c_count:
            klassen_cpp["4"]["x"].append(np.log10(c_count))
            klassen_cpp["4"]["y"].append(np.log10(cpp_count))
        #flafinder
        if ff_count == 0:        
            klassen_ff["1"]["x"].append(np.log10(c_count))
            klassen_ff["1"]["y"].append(-1)
        elif ff_count == c_count:
            klassen_ff["2"]["x"].append(np.log10(c_count))
            klassen_ff["2"]["y"].append(np.log10(ff_count))
        elif ff_count < c_count:
            klassen_ff["3"]["x"].append(np.log10(c_count))
            klassen_ff["3"]["y"].append(np.log10(ff_count))
        elif ff_count > c_count:
            klassen_ff["4"]["x"].append(np.log10(c_count))
            klassen_ff["4"]["y"].append(np.log10(ff_count))
            
    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    plt.xlabel("Anzahl Commits", fontsize=35,fontweight='bold')
    # plt.ylabel("Zeit zwischen ersten und letzten Commits des Committers in Tagen")
    plt.ylabel("Anzahl Fehler", fontsize=35,fontweight='bold')
    # for x,y in zip(x_plt, y_plt):
    #     plt.scatter(x, y, alpha=0.1)
    # for x,y, color in zip(x_plt, y_plt, colors):
    #     for col in color:
    #         plt.scatter(x, y, alpha=0.1, color=col)
    plt.xticks([ 0,0.7, 1, 2, 3, 4],[1,5, 10, 100, 1000, 10000])
    plt.yticks([-1,0, 0.7, 1, 2, 3, 4],[0,1,5, 10, 100, 1000, 10000])
    plt.scatter(klassen_cpp["1"]["x"], klassen_cpp["1"]["y"], alpha=0.3, color = "green")
    plt.scatter(klassen_cpp["2"]["x"], klassen_cpp["2"]["y"], alpha=0.3, color = "blue")
    plt.scatter(klassen_cpp["3"]["x"], klassen_cpp["3"]["y"], alpha=0.3, color = "orange")
    plt.scatter(klassen_cpp["4"]["x"], klassen_cpp["4"]["y"], alpha=0.3, color = "red")
    plt.savefig("FRAGE_12_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    plt.xlabel("Anzahl Commits", fontsize=35,fontweight='bold')
    # plt.ylabel("Zeit zwischen ersten und letzten Commits des Committers in Tagen")
    plt.ylabel("Anzahl Fehler", fontsize=35,fontweight='bold')
    # for x,y in zip(x_plt, y_plt):
    #     plt.scatter(x, y, alpha=0.1)
    # for x,y, color in zip(x_plt, y_plt, colors):
    #     for col in color:
    #         plt.scatter(x, y, alpha=0.1, color=col)
    plt.xticks([ 0,0.7, 1, 2, 3, 4],[1,5, 10, 100, 1000, 10000])
    plt.yticks([-1,0, 0.7, 1, 2, 3, 4],[0,1,5, 10, 100, 1000, 10000])
    plt.scatter(klassen_ff["1"]["x"], klassen_ff["1"]["y"], alpha=0.3, color = "green")
    plt.scatter(klassen_ff["2"]["x"], klassen_ff["2"]["y"], alpha=0.3, color = "blue")
    plt.scatter(klassen_ff["3"]["x"], klassen_ff["3"]["y"], alpha=0.3, color = "orange")
    plt.scatter(klassen_ff["4"]["x"], klassen_ff["4"]["y"], alpha=0.3, color = "red")
    plt.savefig("FRAGE_12_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    punkte_1_cpp = [np.array((x, y)) for x, y in zip(klassen_cpp["1"]["x"], klassen_cpp["1"]["y"])]
    punkte_2_cpp = [np.array((x, y)) for x, y in zip(klassen_cpp["2"]["x"], klassen_cpp["2"]["y"])]
    punkte_3_cpp = [np.array((x, y)) for x, y in zip(klassen_cpp["3"]["x"], klassen_cpp["3"]["y"])]
    punkte_4_cpp = [np.array((x, y)) for x, y in zip(klassen_cpp["4"]["x"], klassen_cpp["4"]["y"])]

    combis_1_cpp = list(combinations(punkte_1_cpp, 2))
    combis_2_cpp = list(combinations(punkte_2_cpp, 2))
    combis_3_cpp = list(combinations(punkte_3_cpp, 2))
    combis_4_cpp = list(combinations(punkte_4_cpp, 2))

    dist_1_cpp = []
    dist_2_cpp = []
    dist_3_cpp = []
    dist_4_cpp = []

    for combi in combis_1_cpp:
        dist_1_cpp.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_2_cpp:
        dist_2_cpp.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_3_cpp:
        dist_3_cpp.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_4_cpp:
        dist_4_cpp.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))

    punkte_1_ff = [np.array((x, y)) for x, y in zip(klassen_ff["1"]["x"], klassen_ff["1"]["y"])]
    punkte_2_ff = [np.array((x, y)) for x, y in zip(klassen_ff["2"]["x"], klassen_ff["2"]["y"])]
    punkte_3_ff = [np.array((x, y)) for x, y in zip(klassen_ff["3"]["x"], klassen_ff["3"]["y"])]
    punkte_4_ff = [np.array((x, y)) for x, y in zip(klassen_ff["4"]["x"], klassen_ff["4"]["y"])]

    combis_1_ff = list(combinations(punkte_1_ff, 2))
    combis_2_ff = list(combinations(punkte_2_ff, 2))
    combis_3_ff = list(combinations(punkte_3_ff, 2))
    combis_4_ff = list(combinations(punkte_4_ff, 2))

    dist_1_ff = []
    dist_2_ff= []
    dist_3_ff = []
    dist_4_ff = []

    for combi in combis_1_ff:
        dist_1_ff.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_2_ff:
        dist_2_ff.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_3_ff:
        dist_3_ff.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    for combi in combis_4_ff:
        dist_4_ff.append(math.sqrt(((combi[1][0]-combi[0][0])**2)+((combi[1][1]-combi[0][1])**2)))
    print("Cppcheck ------------------------------------------------")
    print("hSUM(K1) = ",sum(dist_1_cpp),"\nhSCHNITT(K1) = ", sum(dist_1_cpp)/(0.5*len(combis_1_cpp)*(len(combis_1_cpp)-1)), "cSchnitt = ", (0.5*len(combis_1_cpp)*(len(combis_1_cpp)-1)), "\n\n")
    print("hSUM(K2) = ",sum(dist_2_cpp),"\nhSCHNITT(K1) = ", sum(dist_2_cpp)/(0.5*len(combis_2_cpp)*(len(combis_2_cpp)-1)), "cSchnitt = ", (0.5*len(combis_2_cpp)*(len(combis_2_cpp)-1)), "\n\n")
    print("hSUM(K3) = ",sum(dist_3_cpp),"\nhSCHNITT(K1) = ", sum(dist_3_cpp)/(0.5*len(combis_3_cpp)*(len(combis_3_cpp)-1)), "cSchnitt = ", (0.5*len(combis_3_cpp)*(len(combis_3_cpp)-1)), "\n\n")
    print("hSUM(K4) = ",sum(dist_4_cpp),"\nhSCHNITT(K1) = ", sum(dist_4_cpp)/(0.5*len(combis_4_cpp)*(len(combis_4_cpp)-1)), "cSchnitt = ", (0.5*len(combis_4_cpp)*(len(combis_4_cpp)-1)), "\n\n")
    print("Flawfinder ------------------------------------------------")
    print("hSUM(K1) = ",sum(dist_1_ff),"\nhSCHNITT(K1) = ", sum(dist_1_ff)/(0.5*len(combis_1_ff)*(len(combis_1_ff)-1)), "cSchnitt = ", (0.5*len(combis_1_ff)*(len(combis_1_ff)-1)), "\n\n")
    print("hSUM(K2) = ",sum(dist_2_ff),"\nhSCHNITT(K1) = ", sum(dist_2_ff)/(0.5*len(combis_2_ff)*(len(combis_2_ff)-1)), "cSchnitt = ", (0.5*len(combis_2_ff)*(len(combis_2_ff)-1)), "\n\n")
    print("hSUM(K3) = ",sum(dist_3_ff),"\nhSCHNITT(K1) = ", sum(dist_3_ff)/(0.5*len(combis_3_ff)*(len(combis_3_ff)-1)), "cSchnitt = ", (0.5*len(combis_3_ff)*(len(combis_3_ff)-1)), "\n\n")
    print("hSUM(K4) = ",sum(dist_4_ff),"\nhSCHNITT(K1) = ", sum(dist_4_ff)/(0.5*len(combis_4_ff)*(len(combis_4_ff)-1)), "cSchnitt = ", (0.5*len(combis_4_ff)*(len(combis_4_ff)-1)), "\n\n")
    global h_schnitt_cppcheck_summe
    global h_schnitt_flawfinder_summe
    global h_summe_cppcheck_summe
    global h_summe_flawfinder_summe
    h_summe_cppcheck_summe = sum(dist_1_cpp)+sum(dist_2_cpp)+sum(dist_3_cpp)+sum(dist_4_cpp)
    h_summe_flawfinder_summe = sum(dist_1_ff)+sum(dist_2_ff)+sum(dist_3_ff)+sum(dist_4_ff)
    h_schnitt_cppcheck_summe = sum(dist_1_cpp)/(0.5*len(combis_1_cpp)*(len(combis_1_cpp)-1))+sum(dist_2_cpp)/(0.5*len(combis_2_cpp)*(len(combis_2_cpp)-1))+sum(dist_3_cpp)/(0.5*len(combis_3_cpp)*(len(combis_3_cpp)-1))+sum(dist_4_cpp)/(0.5*len(combis_4_cpp)*(len(combis_4_cpp)-1))
    h_schnitt_flawfinder_summe = sum(dist_1_ff)/(0.5*len(combis_1_ff)*(len(combis_1_ff)-1))+sum(dist_2_ff)/(0.5*len(combis_2_ff)*(len(combis_2_ff)-1))+sum(dist_3_ff)/(0.5*len(combis_3_ff)*(len(combis_3_ff)-1))+sum(dist_4_ff)/(0.5*len(combis_4_ff)*(len(combis_4_ff)-1))
# frage12()

In [ ]:
def frage12_single_complete():
    autoren = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(id) from commits
    group by author''')
    for a, count in mycursor:
        autoren[str(a)] = {"count_commits": count, "count_cpp": 0, "count_ff":0}
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(c_fehler) from
    (select  commits.author as author , cppcheck.fehler as c_fehler, categorie
    from cppcheck
    join commits
    where commits.id = cppcheck.commit_id) as a
    where categorie = "error"
    group by author''')
    for a, count in mycursor:
        autoren[str(a)]["count_cpp"] = count
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select author, count(c_fehler) from
    (select  commits.author as author , flawfinder.name as c_fehler, Level
    from flawfinder
    join commits
    where commits.id = flawfinder.commit_id) as a
    where Level > 2
    group by author''')
    for a, count in mycursor:
        autoren[str(a)]["count_ff"] = count
    mycursor.close()

    x_plt = [autoren[str(x)]["count_commits"] for x in autoren]
    y_plt_cpp = []
    for x in autoren:
        y_plt_cpp.append(autoren[str(x)]["count_cpp"])
    # y_plt_cpp = [autoren[str(x)]["count_cpp"] for x in autoren]
    # y_plt_ff = [autoren[str(x)]["count_ff"] for x in autoren]
    y_plt_ff = []
    for x in autoren:
        y_plt_ff.append(autoren[str(x)]["count_ff"])

    klassen_cpp = {"1": {"x":[], "y":[]}, "2": {"x":[], "y":[]}, "3": {"x":[], "y":[]}, "4": {"x":[], "y":[]}}
    klassen_ff = {"1": {"x":[], "y":[]}, "2": {"x":[], "y":[]}, "3": {"x":[], "y":[]}, "4": {"x":[], "y":[]}}
    # for c_count, cpp  _count in y_plt_cpp:
    for item in autoren:
        c_count = autoren[str(item)]["count_commits"]
        cpp_count = autoren[str(item)]["count_cpp"]
        ff_count = autoren[str(item)]["count_ff"]
        #cppcheck
        if cpp_count == 0:        
            klassen_cpp["1"]["x"].append(c_count)
            klassen_cpp["1"]["y"].append(cpp_count)
        elif cpp_count == c_count:
            klassen_cpp["2"]["x"].append(c_count)
            klassen_cpp["2"]["y"].append(cpp_count)
        elif cpp_count < c_count:
            klassen_cpp["3"]["x"].append(c_count)
            klassen_cpp["3"]["y"].append(cpp_count)
        elif cpp_count > c_count:
            klassen_cpp["4"]["x"].append(c_count)
            klassen_cpp["4"]["y"].append(cpp_count)
        #flafinder
        if ff_count == 0:        
            klassen_ff["1"]["x"].append(c_count)
            klassen_ff["1"]["y"].append(ff_count)
        elif ff_count == c_count:
            klassen_ff["2"]["x"].append(c_count)
            klassen_ff["2"]["y"].append(ff_count)
        elif ff_count < c_count:
            klassen_ff["3"]["x"].append(c_count)
            klassen_ff["3"]["y"].append(ff_count)
        elif ff_count > c_count:
            klassen_ff["4"]["x"].append(c_count)
            klassen_ff["4"]["y"].append(ff_count)

    punkte_1_cpp = [[x, y] for x, y in zip(klassen_cpp["1"]["x"], klassen_cpp["1"]["y"])]
    punkte_2_cpp = [[x, y] for x, y in zip(klassen_cpp["2"]["x"], klassen_cpp["2"]["y"])]
    punkte_3_cpp = [[x, y] for x, y in zip(klassen_cpp["3"]["x"], klassen_cpp["3"]["y"])]
    punkte_4_cpp = [[x, y] for x, y in zip(klassen_cpp["4"]["x"], klassen_cpp["4"]["y"])]

    punkte_1_2_cpp = [punkte_1_cpp, punkte_2_cpp] 
    punkte_1_3_cpp = [punkte_1_cpp, punkte_3_cpp] 
    punkte_1_4_cpp = [punkte_1_cpp, punkte_4_cpp] 
    punkte_2_3_cpp = [punkte_2_cpp, punkte_3_cpp] 
    punkte_2_4_cpp = [punkte_2_cpp, punkte_4_cpp] 
    punkte_3_4_cpp = [punkte_3_cpp, punkte_4_cpp] 

    combis_1_2_cpp = list(itertools.product(*punkte_1_2_cpp))
    combis_1_3_cpp = list(itertools.product(*punkte_1_3_cpp))
    combis_1_4_cpp = list(itertools.product(*punkte_1_4_cpp))
    combis_2_3_cpp = list(itertools.product(*punkte_2_3_cpp))
    combis_2_4_cpp = list(itertools.product(*punkte_2_4_cpp))
    combis_3_4_cpp = list(itertools.product(*punkte_3_4_cpp))

    dist_1_2_cpp = []
    dist_1_3_cpp = []
    dist_1_4_cpp = []
    dist_2_3_cpp = []
    dist_2_4_cpp = []
    dist_3_4_cpp = []

    for combi in combis_1_2_cpp:
        dist_1_2_cpp.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_1_3_cpp:
        dist_1_3_cpp.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_1_4_cpp:
        dist_1_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_2_3_cpp:
        dist_2_3_cpp.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_2_4_cpp:
        dist_2_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_3_4_cpp:
        dist_3_4_cpp.append(distance.euclidean(combi[0], combi[1]))
    print("-------------------CPPCHECK-------------------------------")
    print("v_single_cpp(K_1,K_2) = ",min(dist_1_2_cpp))
    print("v_complete_cpp(K_1,K_2) = ",max(dist_1_2_cpp))
    print("v_single_cpp(K_1,K_3) = ",min(dist_1_3_cpp))
    print("v_complete_cpp(K_1,K_3) = ",max(dist_1_3_cpp))
    print("v_single_cpp(K_1,K_4) = ",min(dist_1_4_cpp))
    print("v_complete_cpp(K_1,K_4) = ",max(dist_1_4_cpp))
    print("v_single_cpp(K_2,K_3) = ",min(dist_2_3_cpp))
    print("v_complete_cpp(K_2,K_3) = ",max(dist_2_3_cpp))
    print("v_single_cpp(K_2,K_4) = ",min(dist_2_4_cpp))
    print("v_complete_cpp(K_2,K_4) = ",max(dist_2_4_cpp))
    print("v_single_cpp(K_3,K_4) = ",min(dist_3_4_cpp))
    print("v_complete_cpp(K_3,K_4) = ",max(dist_3_4_cpp))

    punkte_1_ff = [[x, y] for x, y in zip(klassen_ff["1"]["x"], klassen_ff["1"]["y"])]
    punkte_2_ff = [[x, y] for x, y in zip(klassen_ff["2"]["x"], klassen_ff["2"]["y"])]
    punkte_3_ff = [[x, y] for x, y in zip(klassen_ff["3"]["x"], klassen_ff["3"]["y"])]
    punkte_4_ff = [[x, y] for x, y in zip(klassen_ff["4"]["x"], klassen_ff["4"]["y"])]

    punkte_1_2_ff = [punkte_1_ff, punkte_2_ff] 
    punkte_1_3_ff = [punkte_1_ff, punkte_3_ff] 
    punkte_1_4_ff = [punkte_1_ff, punkte_4_ff] 
    punkte_2_3_ff = [punkte_2_ff, punkte_3_ff] 
    punkte_2_4_ff = [punkte_2_ff, punkte_4_ff] 
    punkte_3_4_ff = [punkte_3_ff, punkte_4_ff] 

    combis_1_2_ff = list(itertools.product(*punkte_1_2_ff))
    combis_1_3_ff = list(itertools.product(*punkte_1_3_ff))
    combis_1_4_ff = list(itertools.product(*punkte_1_4_ff))
    combis_2_3_ff = list(itertools.product(*punkte_2_3_ff))
    combis_2_4_ff = list(itertools.product(*punkte_2_4_ff))
    combis_3_4_ff = list(itertools.product(*punkte_3_4_ff))

    dist_1_2_ff = []
    dist_1_3_ff = []
    dist_1_4_ff = []
    dist_2_3_ff = []
    dist_2_4_ff = []
    dist_3_4_ff = []

    for combi in combis_1_2_ff:
        dist_1_2_ff.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_1_3_ff:
        dist_1_3_ff.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_1_4_ff:
        dist_1_4_ff.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_2_3_ff:
        dist_2_3_ff.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_2_4_ff:
        dist_2_4_ff.append(distance.euclidean(combi[0], combi[1]))
    for combi in combis_3_4_ff:
        dist_3_4_ff.append(distance.euclidean(combi[0], combi[1]))
    print("-------------------FLAWFINDER-------------------------------")
    print("v_single_ff(K_1,K_2) = ",min(dist_1_2_ff))
    print("v_complete_ff(K_1,K_2) = ",max(dist_1_2_ff))
    print("v_single_ff(K_1,K_3) = ",min(dist_1_3_ff))
    print("v_complete_ff(K_1,K_3) = ",max(dist_1_3_ff))
    print("v_single_ff(K_1,K_4) = ",min(dist_1_4_ff))
    print("v_complete_ff(K_1,K_4) = ",max(dist_1_4_ff))
    print("v_single_ff(K_2,K_3) = ",min(dist_2_3_ff))
    print("v_complete_ff(K_2,K_3) = ",max(dist_2_3_ff))
    print("v_single_ff(K_2,K_4) = ",min(dist_2_4_ff))
    print("v_complete_ff(K_2,K_4) = ",max(dist_2_4_ff))
    print("v_single_ff(K_3,K_4) = ",min(dist_3_4_ff))
    print("v_complete_ff(K_3,K_4) = ",max(dist_3_4_ff))


    print("----------------------LATEX CPPCHECK SINGLE--------------------------------")
    print("$v_{Single,Cppcheck}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_2_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_3_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_4_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_3_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_4_cpp),"$","\\","\\")
    print("$v_{Single,Cppcheck}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_3_4_cpp),"$","\\","\\")

    print("----------------------LATEX CPPCHECK COMPLETE--------------------------------")
    print("$v_{Complete,Cppcheck}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_2_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_3_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_4_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_3_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_4_cpp),"$","\\","\\")
    print("$v_{Complete,Cppcheck}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_3_4_cpp),"$","\\","\\")

    print("----------------------LATEX FLAWFINDER SINGLE--------------------------------")
    print("$v_{Single,Flawfinder}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_2_ff),"$","\\","\\")
    print("$v_{Single,Flawfinder}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_3_ff),"$","\\","\\")
    print("$v_{Single,Flawfinder}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_1_4_ff),"$","\\","\\")
    print("$v_{Single,Flawfinder}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_3_ff),"$","\\","\\")
    print("$v_{Single,Flawfinder}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_2_4_ff),"$","\\","\\")
    print("$v_{Single,Flawfinder}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","min} d(i,j) = ",min(dist_3_4_ff),"$","\\","\\")

    print("----------------------LATEX FLAWFINDER COMPLETE--------------------------------")
    print("$v_{Complete,Flawfinder}(K_1, K_2) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_2_ff),"$","\\","\\")
    print("$v_{Complete,Flawfinder}(K_1, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_3_ff),"$","\\","\\")
    print("$v_{Complete,Flawfinder}(K_1, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_1_4_ff),"$","\\","\\")
    print("$v_{Complete,Flawfinder}(K_2, K_3) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_3_ff),"$","\\","\\")
    print("$v_{Complete,Flawfinder}(K_2, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_2_4_ff),"$","\\","\\")
    print("$v_{Complete,Flawfinder}(K_3, K_4) = ","\\","underset{i ","\\","in K,j ","\\","in K}{","\\","max} d(i,j) = ",max(dist_3_4_ff),"$","\\","\\")

    print("---------Bewertung Cppcheck-----------------------")
    print("Kombination aus h(K)Schnitt,Cppcheck und vSingle,Cppcheck(K1, K2): ", h_schnitt_cppcheck_summe, "/", sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]), "=", h_schnitt_cppcheck_summe/sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]))
    print("Kombination aus hSumme,Cppcheck(K) und vComplete,Cppcheck(K1, K2): ", h_summe_cppcheck_summe, "/",sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]), "=", h_summe_cppcheck_summe/sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]))
    print("---------Bewertung Flawfinder-----------------------")
    print("Kombination aus h(K)Schnitt,Flawfinder und vSingle,Flawfinder(K1, K2): ", h_schnitt_cppcheck_summe,"/",sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]) , "=",h_schnitt_cppcheck_summe/sum([min(dist_1_2_cpp), min(dist_1_3_cpp), min(dist_1_4_cpp), min(dist_2_3_cpp), min(dist_2_4_cpp), min(dist_3_4_cpp)]))
    print("Kombination aus hSumme,Flawfinder(K) und vComplete,Flawfinder(K1, K2):", h_summe_flawfinder_summe,"/",sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]), "=",h_summe_flawfinder_summe/sum([max(dist_1_2_cpp), max(dist_1_3_cpp), max(dist_1_4_cpp), max(dist_2_3_cpp), max(dist_2_4_cpp), max(dist_3_4_cpp)]))

# frage12_single_complete()

### Frage 13: Besteht ein linearer Zusammenhang zwischen Couplings und der Anzahl Fehler?

In [ ]:
def frage13_korrkoef():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(flawfinder.commit_id), concat(file_name1, file_name2), file_name1, file_name2, count from flawfinder
    join
    (select commit_id, file_name1, file_name2, count from stats 
    join
    (select file_name1, file_name2, count from couplings
    where count > 1000) as coups 
    where stats.file_name = coups.file_name1 or stats.file_name = coups.file_name2) as stats_coups
    where stats_coups.commit_id = flawfinder.commit_id and flawfinder.Level >2
    group by concat(file_name1, file_name2)''')
    bugs_ff = []
    coups_ff = []
    tabelle_ff = []
    for bug_count_ff, files, file1, file2, coup_count_ff in mycursor:
        bugs_ff.append(bug_count_ff)
        coups_ff.append(coup_count_ff)
        tabelle_ff.append([file1, file2, coup_count_ff, bug_count_ff])
    mycursor.close() 
    print("Der Korrelationskoeffizient zwischen den Couplings und den Schwachstellen nach Flawfinder beträgt:\n")
    print(np.corrcoef(bugs_ff, coups_ff))

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(cppcheck.commit_id), concat(file_name1, file_name2), file_name1, file_name2, count from cppcheck
    join
    (select commit_id, file_name1, file_name2, count from stats 
    join
    (select file_name1, file_name2, count from couplings
    where count > 1000) as coups 
    where stats.file_name = coups.file_name1 or stats.file_name = coups.file_name2) as stats_coups
    where stats_coups.commit_id = cppcheck.commit_id and cppcheck.categorie ="error"
    group by concat(file_name1, file_name2)''')
    bugs_cpp = []
    coups_cpp = []
    tabelle_cpp = []
    for bug_count_cpp, files, file1, file2, coup_count_cpp in mycursor:
        bugs_cpp.append(bug_count_cpp)
        coups_cpp.append(coup_count_cpp)
        tabelle_cpp.append([file1, file2, coup_count_cpp, bug_count_cpp])
    mycursor.close() 
    print("Der Korrelationskoeffizient zwischen den Couplings und den Fehlern nach Cppcheck beträgt:\n")
    print(np.corrcoef(bugs_cpp, coups_cpp))


    print("---------------------------Latex Tabelle Flawfinder--------------------------------------")
    print(tabulate(tabelle_ff, headers=["Pfad1", "Pfad2", "Anzahl Couplings", "Anzahl potentieller Schwachstellen in diesen Commits"], tablefmt='latex'))
    print("---------------------------Latex Tabelle Cppcheck--------------------------------------")
    print(tabulate(tabelle_cpp, headers=["Pfad1", "Pfad2", "Anzahl Couplings", "Anzahl Fehler in diesen Commits"], tablefmt='latex'))
    print("---------------------------Latex Tabelle Zusammengefasst--------------------------------------")
    tabelle_gesamt = [[zahl_id+1, tabelle_cpp[zahl_id][2], tabelle_cpp[zahl_id][3], tabelle_ff[zahl_id][3]] for zahl_id in range(83)]
    print(tabulate(tabelle_gesamt, headers=["Coupling Identifikationsnummer", "Anzahl Couplings", "Cppcheck: Anzahl der kritischen Meldungen in diesen Commits", "Flawfinder: Anzahl der kritischen Meldungen in diesen Commits"], tablefmt='latex'))

# frage13_korrkoef()

In [ ]:
def frage13_abbildungen():
    ## flawfinder
    flawfinder = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select concat(file_name1, file_name2), count, categorie from 
        (select file_name1, file_name2, count, categorie from couplings
        join files
        where files.file_name = couplings.file_name1 or files.file_name = couplings.file_name2) as a
        where count > 1000''')
    for files_sql, count_coupling, categorie in mycursor:
        if files_sql in flawfinder.keys():
            flawfinder[str(files_sql)]["categorie2"] = categorie
        else:
            flawfinder[str(files_sql)] = {"coupling_count": count_coupling, "categorie1": categorie, "bug_cpp":0}
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(flawfinder.commit_id), concat(file_name1, file_name2) from flawfinder
     join
     (select commit_id, file_name1, file_name2, count from stats 
     join
     (select file_name1, file_name2, count from couplings
     where count > 1000) as coups 
     where stats.file_name = coups.file_name1 or stats.file_name = coups.file_name2) as stats_coups
     where stats_coups.commit_id = flawfinder.commit_id and flawfinder.Level >2
     group by concat(file_name1, file_name2)''')
    for bug_count_cpp, files_sql in mycursor:
        flawfinder[str(files_sql)]["bug_cpp"]=bug_count_cpp
    mycursor.close() 
    both_quelltext_x = []
    one_quelltext_x = []
    no_quelltext_x = [] 
    both_quelltext_y = []
    one_quelltext_y = []
    no_quelltext_y = []
    for item in flawfinder:
        if flawfinder[str(item)]["categorie1"] == "Quellcode" and flawfinder[str(item)]["categorie2"] == "Quellcode":
            both_quelltext_x.append(flawfinder[str(item)]["coupling_count"])
            both_quelltext_y.append(flawfinder[str(item)]["bug_cpp"])
        elif flawfinder[str(item)]["categorie1"] == "Quellcode" or flawfinder[str(item)]["categorie2"] == "Quellcode":
            one_quelltext_x.append(flawfinder[str(item)]["coupling_count"])
            one_quelltext_y.append(flawfinder[str(item)]["bug_cpp"])
        else:
            no_quelltext_x.append(flawfinder[str(item)]["coupling_count"])
            no_quelltext_y.append(flawfinder[str(item)]["bug_cpp"])

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    plt.xlim(1100,3100)
    plt.ylim(800,8600)
    plt.xlabel("Anzahl Fehler in Coupling", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Auftritte des Coupling", fontsize=35,fontweight='bold')
    plt.scatter(both_quelltext_y, both_quelltext_x, color = "red", label="Beides Quelltext Dateien")
    plt.scatter(one_quelltext_y, one_quelltext_x, color = "orange",  label="Eine Quelltext Datei")
    plt.scatter(no_quelltext_y, no_quelltext_x, color = "green",  label="Keine Quelltext Dateien")
    plt.legend(fontsize=15)
    plt.savefig("FRAGE_13_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    ## cppcheck
    cppcheck = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select concat(file_name1, file_name2), count, categorie from 
        (select file_name1, file_name2, count, categorie from couplings
        join files
        where files.file_name = couplings.file_name1 or files.file_name = couplings.file_name2) as a
        where count > 1000''')
    for files_sql, count_coupling, categorie in mycursor:
        if files_sql in cppcheck.keys():
            cppcheck[str(files_sql)]["categorie2"] = categorie
        else:
            cppcheck[str(files_sql)] = {"coupling_count": count_coupling, "categorie1": categorie, "bug_cpp":0}
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(cppcheck.commit_id), concat(file_name1, file_name2) from cppcheck
        join
        (select commit_id, file_name1, file_name2, count from stats 
        join
        (select file_name1, file_name2, count from couplings
        where count > 1000) as coups 
        where stats.file_name = coups.file_name1 or stats.file_name = coups.file_name2) as stats_coups
        where stats_coups.commit_id = cppcheck.commit_id and cppcheck.categorie ="error"
        group by concat(file_name1, file_name2)''')
    for bug_count_cpp, files_sql in mycursor:
        cppcheck[str(files_sql)]["bug_cpp"]=bug_count_cpp
    mycursor.close() 
    both_quelltext_x = []
    one_quelltext_x = []
    no_quelltext_x = [] 
    both_quelltext_y = []
    one_quelltext_y = []
    no_quelltext_y = []
    for item in cppcheck:
        if cppcheck[str(item)]["categorie1"] == "Quellcode" and cppcheck[str(item)]["categorie2"] == "Quellcode":
            both_quelltext_x.append(cppcheck[str(item)]["coupling_count"])
            both_quelltext_y.append(cppcheck[str(item)]["bug_cpp"])
        elif cppcheck[str(item)]["categorie1"] == "Quellcode" or cppcheck[str(item)]["categorie2"] == "Quellcode":
            one_quelltext_x.append(cppcheck[str(item)]["coupling_count"])
            one_quelltext_y.append(cppcheck[str(item)]["bug_cpp"])
        else:
            no_quelltext_x.append(cppcheck[str(item)]["coupling_count"])
            no_quelltext_y.append(cppcheck[str(item)]["bug_cpp"])

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    plt.xlim(6300,35000)
    plt.ylim(800,8600)
    plt.xlabel("Anzahl Fehler in Coupling", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Auftritte des Coupling", fontsize=35,fontweight='bold')
    plt.scatter(both_quelltext_y, both_quelltext_x, color = "red", label="Beides Quelltext Dateien")
    plt.scatter(one_quelltext_y, one_quelltext_x, color = "orange",  label="Eine Quelltext Datei")
    plt.scatter(no_quelltext_y, no_quelltext_x, color = "green",  label="Keine Quelltext Dateien")
    plt.legend(fontsize=15)
    plt.savefig("FRAGE_13_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

frage13_abbildungen()

### Frage 14: Besteht ein linearer Zusammenhang zwischen den Anzahl der veränderten Dateien und der Anzahl Fehler?

In [ ]:
def frage14():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(file_name) from stats
    group by commit_id''')
    files_count = {}
    for commit_id, count in mycursor:
        files_count[str(commit_id)] = {"count_files":count, "cppcheck_mit": 0, "cppcheck_ohne": 0 , "flawfinder": 0, "alle":0}
    mycursor.close() 


    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(fehler) from cppcheck
    where categorie != "error"
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["cppcheck_mit"] = count
    mycursor.close() 
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(fehler) from cppcheck
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["alle"] = count
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(fehler) from cppcheck
    where categorie="error"
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["cppcheck_ohne"] = count
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(file_name) from flawfinder where Level > 2
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["flawfinder"] = count
    mycursor.close() 

        
    counti_files = [files_count[x]["count_files"] for x in files_count]
    counti_cpp_with = [files_count[x]["cppcheck_mit"] for x in files_count]
    alle = [files_count[x]["alle"] for x in files_count]
    counti_cpp_ohne = [files_count[x]["cppcheck_ohne"] for x in files_count]
    counti_flawfinder = [files_count[x]["flawfinder"] for x in files_count]

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    # plt.xlim(6300,35000)
    # plt.ylim(800,8600)
    plt.xlabel("Anzahl Dateien", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, alle)", fontsize=35,fontweight='bold')
    plt.scatter(counti_files, counti_cpp_with, color="green", label ="Information, Warnung, Performanz,\nPortabilität, Stil", alpha=0.2)
    plt.scatter(counti_files, counti_cpp_ohne, color="red", label ="Error", alpha=0.2)
    leg = plt.legend(fontsize=15, loc=1)
    leg.set_title('Kategorie',prop={'size':18})
    plt.savefig("FRAGE_14_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    # plt.xlim(6300,35000)
    # plt.ylim(800,8600)
    plt.xlabel("Anzahl Dateien", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, Error)", fontsize=35,fontweight='bold')
    plt.scatter(counti_files, counti_cpp_ohne, color="red", alpha=0.2)
    # plt.legend(fontsize=15)
    plt.savefig("FRAGE_14_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(15, 15)
    # plt.xlim(6300,35000)
    # plt.ylim(800,8600)
    plt.xlabel("Anzahl Dateien", fontsize=35,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Flawfinder, Risikolevel > 2)", fontsize=35,fontweight='bold')
    plt.scatter(counti_files, counti_flawfinder, alpha=0.2)
    # plt.legend(fontsize=15)
    plt.savefig("FRAGE_14_3.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    # faults and files
    correlationfiles = np.corrcoef(counti_files, alle)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Dateien und gefundene Fehler (Cppcheck mit allen Kategorie): \n",correlationfiles)
    # # faults and ins
    correlationins = np.corrcoef(counti_files, counti_cpp_ohne)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Dateien und gefundene Fehler (Cppcheck nur error): \n",correlationins)
    # # faults and dels
    correlationdels = np.corrcoef(counti_files, counti_flawfinder)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Dateien und gefundene Fehler (Flawfinder Level > 2): \n",correlationdels)

# frage14()

### Frage 15: Besteht ein linearer Zusammenhang zwischen den Anzahl der veränderten Zeilen und der Anzahl Fehler?

In [ ]:
def frage15():
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, sum(insertions), sum(deletions) from stats
    group by commit_id''')
    files_count = {}
    for commit_id, inss, delss in mycursor:
        files_count[str(commit_id)] = {"count_ins":int(inss),"count_dels":int(delss), "cppcheck_mit": 0, "cppcheck_ohne": 0 , "flawfinder": 0}
    mycursor.close() 


    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(fehler) from cppcheck
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["cppcheck_mit"] = int(count)
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(fehler) from cppcheck
    where categorie="error"
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["cppcheck_ohne"] = int(count)
    mycursor.close() 

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select commit_id, count(file_name) from flawfinder where Level > 2
    group by commit_id''')
    for commit_id, count in mycursor:
        if commit_id != 25569:
            files_count[str(commit_id)]["flawfinder"] = int(count)
    mycursor.close() 

            
    counti_ins = [files_count[x]["count_ins"] for x in files_count]
    counti_dels = [files_count[x]["count_dels"] for x in files_count]
    counti_total = [files_count[x]["count_dels"]+files_count[x]["count_ins"] for x in files_count]
    counti_cpp_with = [files_count[x]["cppcheck_mit"] for x in files_count]
    counti_cpp_ohne = [files_count[x]["cppcheck_ohne"] for x in files_count]
    counti_flawfinder = [files_count[x]["flawfinder"] for x in files_count]

    # -------------hinzugefügt--------------------------
    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl hinzugefügte Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, alle)", fontsize=15,fontweight='bold')
    plt.scatter(counti_ins, counti_cpp_with, alpha=0.2)
    # plt.plot([0,2500000],[0, 0.2445*25000], color="red")
    plt.savefig("FRAGE_15_1.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl hinzugefügte Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, Error)", fontsize=15,fontweight='bold')
    plt.scatter(counti_ins, counti_cpp_ohne, alpha=0.2)
    # plt.plot([0,2500000],[0, 0.385*1000], color="red")
    plt.savefig("FRAGE_15_2.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl hinzugefügte Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Flawfinder)", fontsize=15,fontweight='bold')
    plt.scatter(counti_ins, counti_flawfinder, alpha=0.2)
    # plt.plot([0,2500000],[0, 0.3397*80], color="red")
    plt.savefig("FRAGE_15_3.png",dpi=240, bbox_inches = 'tight')
    plt.show()
    # -------------gelöscht--------------------------
    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl gelöschter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, alle)", fontsize=15,fontweight='bold')
    plt.scatter(counti_dels, counti_cpp_with, alpha=0.2)
    # plt.plot([0,3000000],[0, 0.3253*25000], color="red")
    plt.savefig("FRAGE_15_4.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl gelöschter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, Error)", fontsize=15,fontweight='bold')
    plt.scatter(counti_dels, counti_cpp_ohne, alpha=0.2)
    # plt.plot([0,3000000],[0, 0.228*1000], color="red")
    plt.savefig("FRAGE_15_5.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl gelöschter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Flawfinder)", fontsize=15,fontweight='bold')
    plt.scatter(counti_dels, counti_flawfinder, alpha=0.2)
    # plt.plot([0,3000000],[0, 0.2158*80], color="red")
    plt.savefig("FRAGE_15_6.png",dpi=240, bbox_inches = 'tight')
    plt.show()
    # -------------total geändert--------------------------
    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl veränderter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, alle)", fontsize=15,fontweight='bold')
    plt.scatter(counti_total, counti_cpp_with, alpha=0.2)
    # plt.plot([0,5000000],[0, 0.3056*25000], color="red")
    plt.savefig("FRAGE_15_7.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl veränderter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Cppcheck, Error)", fontsize=15,fontweight='bold')
    plt.scatter(counti_total, counti_cpp_ohne, alpha=0.2)
    # plt.plot([0,5000000],[0, 0.3253*1000], color="red")
    plt.savefig("FRAGE_15_8.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    fig, ax = plt.subplots() 
    fig.set_size_inches(5,5)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel("Anzahl veränderter Zeilen", fontsize=15,fontweight='bold')
    plt.ylabel("Anzahl Fehler (Flawfinder)", fontsize=15,fontweight='bold')
    plt.scatter(counti_total, counti_flawfinder, alpha=0.2)
    # plt.plot([0,5000000],[0, 0.2943*80], color="red")
    plt.savefig("FRAGE_15_9.png",dpi=240, bbox_inches = 'tight')
    plt.show()

    # faults and files
    correlationfiles = np.corrcoef(counti_ins, counti_cpp_with)
    print("Korrelationskoeffizient zwischen Anzahl hinzugefügter Zeilen und gefundene Fehler (Cppcheck mit allen Kategorie): \n",correlationfiles)
    # # faults and ins
    correlationins = np.corrcoef(counti_ins, counti_cpp_ohne)
    print("Korrelationskoeffizient zwischen Anzahl hinzugefügter Zeilen und gefundene Fehler (Cppcheck nur error): \n",correlationins)
    # # faults and dels
    correlationdels = np.corrcoef(counti_ins, counti_flawfinder)
    print("Korrelationskoeffizient zwischen Anzahl hinzugefügter Zeilen und gefundene Fehler (Flawfinder Level > 2): \n",correlationdels)

    correlationfiles = np.corrcoef(counti_dels, counti_cpp_with)
    print("Korrelationskoeffizient zwischen Anzahl gelöschter Zeilen und gefundene Fehler (Cppcheck mit allen Kategorie): \n",correlationfiles)
    # # faults and ins
    correlationins = np.corrcoef(counti_dels, counti_cpp_ohne)
    print("Korrelationskoeffizient zwischen Anzahl gelöschter Zeilen und gefundene Fehler (Cppcheck nur error): \n",correlationins)
    # # faults and dels
    correlationdels = np.corrcoef(counti_dels, counti_flawfinder)
    print("Korrelationskoeffizient zwischen Anzahl gelöschter Zeilen und gefundene Fehler (Flawfinder Level > 2): \n",correlationdels)

    correlationfiles = np.corrcoef(counti_total, counti_cpp_with)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Zeilen und gefundene Fehler (Cppcheck mit allen Kategorie): \n",correlationfiles)
    # # faults and ins
    correlationins = np.corrcoef(counti_total, counti_cpp_ohne)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Zeilen und gefundene Fehler (Cppcheck nur error): \n",correlationins)
    # # faults and dels
    correlationdels = np.corrcoef(counti_total, counti_flawfinder)
    print("Korrelationskoeffizient zwischen Anzahl geänderter Zeilen und gefundene Fehler (Flawfinder Level > 2): \n",correlationdels)

# frage15()

### Frage 16: Wie viele Zeilen werden in der folgenden Woche abhängig von der Anzahl Zeilenänderung der vorigen Woche verändert?

In [ ]:
def frage16():
    counts_per_week_cpp = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select weekofyear(committed_date) as ww, year(committed_date) as y from commits
    group by ww, y''')
    i = 0
    for ww, y in mycursor:
        i += 1
        counts_per_week_cpp[str(ww)+"_"+str(y)] = 0
    mycursor.close()
    print("insegsamt" ,i, "wochen")

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select sum(insertions)+sum(deletions) as summe_zeilen,  weekofyear(committed_date) as ww,year(committed_date) as y from
        (select * from commits 
        join stats
        where stats.commit_id=commits.id) as a
        group by ww, y''')
    week_count = 0
    for count, ww, y in mycursor:
        week_count += 1
        counts_per_week_cpp[str(ww)+"_"+str(y)] = count
    mycursor.close()
    print("insegsamt" ,week_count, "wochen")

    ##sups
    klassen = []
    for week in counts_per_week_cpp:
        if counts_per_week_cpp[str(week)] <= 3000:
            klassen.append(0)
        elif counts_per_week_cpp[str(week)] <= 10000 and counts_per_week_cpp[str(week)] > 3000:
            klassen.append(1)
        elif counts_per_week_cpp[str(week)] <= 30000 and counts_per_week_cpp[str(week)] > 10000:
            klassen.append(2)
        elif counts_per_week_cpp[str(week)] <= 90000 and counts_per_week_cpp[str(week)] > 30000:
            klassen.append(3)
        else:
            klassen.append(4)
    klassen_categorien = ["Sehr Wenig", "Wenig", "Einige", "Viel", "Sehr viel"]
    ##sups (von zu)
    pairs = []
    for i in range(len(klassen)-1):
        paar = [klassen[i], klassen[i+1]]
        if paar[0] == 0 and paar[1] == 0:
            pairs.append("00")
        if paar[0] == 0 and paar[1] == 1:
            pairs.append("01")
        if paar[0] == 0 and paar[1] == 2:
            pairs.append("02")
        if paar[0] == 0 and paar[1] == 3:
            pairs.append("03")
        if paar[0] == 0 and paar[1] == 4:
            pairs.append("04")

        if paar[0] == 1 and paar[1] == 0:
            pairs.append("10")
        if paar[0] == 1 and paar[1] == 1:
            pairs.append("11")
        if paar[0] == 1 and paar[1] == 2:
            pairs.append("12")
        if paar[0] == 1 and paar[1] == 3:
            pairs.append("13")
        if paar[0] == 1 and paar[1] == 4:
            pairs.append("14")

        if paar[0] == 2 and paar[1] == 0:
            pairs.append("20")
        if paar[0] == 2 and paar[1] == 1:
            pairs.append("21")
        if paar[0] == 2 and paar[1] == 2:
            pairs.append("22")
        if paar[0] == 2 and paar[1] == 3:
            pairs.append("23")
        if paar[0] == 2 and paar[1] == 4:
            pairs.append("24")

        if paar[0] == 3 and paar[1] == 0:
            pairs.append("30")
        if paar[0] == 3 and paar[1] == 1:
            pairs.append("31")
        if paar[0] == 3 and paar[1] == 2:
            pairs.append("32")
        if paar[0] == 3 and paar[1] == 3:
            pairs.append("33")
        if paar[0] == 3 and paar[1] == 4:
            pairs.append("34")
        
        if paar[0] == 4 and paar[1] == 0:
            pairs.append("40")
        if paar[0] == 4 and paar[1] == 1:
            pairs.append("41")
        if paar[0] == 4 and paar[1] == 2:
            pairs.append("42")
        if paar[0] == 4 and paar[1] == 3:
            pairs.append("43")
        if paar[0] == 4 and paar[1] == 4:
            pairs.append("44")
    sups_fertig = {str(x):klassen.count(x) for x in klassen}
    sups_two_fertig = {x:pairs.count(x) for x in pairs}

    # for i in range(5):
    #     print("Support(",klassen_categorien[i],") = ", sups_fertig[str(i)], "/",week_count , "=",  round(sups_fertig[str(i)]/week_count*100, 3), "%")
    # print("------------------------------------------------------------------------------------")
    # for i in range(5):
    #     for b in range(5):
    #         try:
    #             print("Support(", klassen_categorien[i], "->", klassen_categorien[b], ") = ", sups_two_fertig[str(i)+str(b)], "/", sups_fertig[str(i)], "=", round(sups_two_fertig[str(i)+str(b)]/sups_fertig[str(i)]*100, 2), "%")
    #         except:
    #             print("Support(", klassen_categorien[i], "->", klassen_categorien[b], ") = 0", "/", sups_fertig[str(i)], "= 0%")
    # print("------------------------------------------------------------------------------------")
    # for i in range(5):
    #     for b in range(5):
    #         try:
    #             print("Conf(", klassen_categorien[i], "->", klassen_categorien[b], "= (", sups_two_fertig[str(i)+str(b)], "/", week_count, ") * (", week_count, "/", sups_fertig[str(i)], ") = ", round(sups_two_fertig[str(i)+str(b)]/sups_fertig[str(i)]*100, 2), "%")
    #         except:
    #             print("Conf(", klassen_categorien[i], "->", klassen_categorien[b], "= (", 0, "/", week_count, ") * (", week_count, "/", sups_fertig[str(i)], ") = 0%")

    print("----------------------LATEX--------------------------------------------------------------")
    for i in range(5):
        print("$sup(",klassen_categorien[i],") = \ frac{", sups_fertig[str(i)], "}{",week_count , "} \ approx",  round(sups_fertig[str(i)]/week_count*100, 3), "\% $ \ \ ")
    print("------------------------------------------------------------------------------------")
    for i in range(5):
        for b in range(5):
            try:
                print("$sup(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", sups_two_fertig[str(i)+str(b)], "}{", week_count, "} \ approx", round(sups_two_fertig[str(i)+str(b)]/(week_count)*100, 2), "\% $ \ \ ")
            except:
                print("$sup(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{0", "}{", week_count, "} = 0\% $ \ \ ")
    print("------------------------------------------------------------------------------------")
    for i in range(5):
        for b in range(5):
            try:
                print("$conf(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", sups_two_fertig[str(i)+str(b)], "}{", week_count, "} \ cdot  \ frac{", week_count, "}{", sups_fertig[str(i)], "} \ approx ", round(sups_two_fertig[str(i)+str(b)]/sups_fertig[str(i)]*100, 2), "\% $ \ \ ")
            except:
                print("$conf(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", 0, "}{", week_count, "} \ cdot  \ frac{", week_count, "}{", sups_fertig[str(i)], "} = 0\% $ \ \ ")

# frage16()

### Frage 17: Führt das Bearbeiten mehrerer Dateien zu mehr Fehlern?

In [ ]:
def frage17():
    counts_per_week_cpp = {}
    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select weekofyear(committed_date) as ww, year(committed_date) as y from commits
    group by ww, y''')
    week_count = 0
    for ww, y in mycursor:
        week_count += 1
        counts_per_week_cpp[str(ww)+"_"+str(y)] = {"cpp": 0, "files": 0}
    mycursor.close()
    print("insgesamt" ,week_count, "wochen")

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(commit_id), weekofyear(committed_date) as ww,year(committed_date) as y from commits 
        join cppcheck
        where cppcheck.commit_id = commits.id and categorie = "error"
        group by ww, y
        order by y, ww''')
    for count, ww, y in mycursor:
        counts_per_week_cpp[str(ww)+"_"+str(y)]["cpp"] = count
    mycursor.close()

    mydb = mysql.connector.connect(host="localhost", user=">USER<", password=">PASSWORT<", database=name)
    mycursor = mydb.cursor()
    mycursor.execute('''select count(file_name), weekofyear(committed_date) as ww,year(committed_date) as y from stats
        join commits
        where commits.id = stats.commit_id
        group by ww, y
        order by y, ww''')
    for count, ww, y in mycursor:
        counts_per_week_cpp[str(ww)+"_"+str(y)]["files"] = count
    mycursor.close()
    klassen_categorien = ["Sehr Wenig", "Wenig", "Einige", "Viel", "Sehr viel"]
    ##sups
    klassen_cpp = []
    for week in counts_per_week_cpp:
        try:
            if counts_per_week_cpp[str(week)]["cpp"] <= 7:
                klassen_cpp.append(0)
            elif counts_per_week_cpp[str(week)]["cpp"] <= 15 and counts_per_week_cpp[str(week)]["cpp"] > 7:
                klassen_cpp.append(1)
            elif counts_per_week_cpp[str(week)]["cpp"] <= 30 and counts_per_week_cpp[str(week)]["cpp"] > 15:
                klassen_cpp.append(2)
            elif counts_per_week_cpp[str(week)]["cpp"] <= 80 and counts_per_week_cpp[str(week)]["cpp"] > 30:
                klassen_cpp.append(3)
            else:
                klassen_cpp.append(4)
        except:
            break

    klassen_files = []
    for week in counts_per_week_cpp:
        try:
            if counts_per_week_cpp[str(week)]["files"] <= 50:
                klassen_files.append(0)
            elif counts_per_week_cpp[str(week)]["files"] <= 90 and counts_per_week_cpp[str(week)]["files"] > 50:
                klassen_files.append(1)
            elif counts_per_week_cpp[str(week)]["files"] <= 150 and counts_per_week_cpp[str(week)]["files"] > 90:
                klassen_files.append(2)
            elif counts_per_week_cpp[str(week)]["files"] <= 350 and counts_per_week_cpp[str(week)]["files"] > 150:
                klassen_files.append(3)
            else:
                klassen_files.append(4)
        except:
            break

    ##sups (von zu)
    pairs = []
    for i in range(len(klassen_cpp)-1):
        paar = [klassen_cpp[i], klassen_files[i]]
        if paar[0] == 0 and paar[1] == 0:
            pairs.append("00")
        if paar[0] == 0 and paar[1] == 1:
            pairs.append("01")
        if paar[0] == 0 and paar[1] == 2:
            pairs.append("02")
        if paar[0] == 0 and paar[1] == 3:
            pairs.append("03")
        if paar[0] == 0 and paar[1] == 4:
            pairs.append("04")

        if paar[0] == 1 and paar[1] == 0:
            pairs.append("10")
        if paar[0] == 1 and paar[1] == 1:
            pairs.append("11")
        if paar[0] == 1 and paar[1] == 2:
            pairs.append("12")
        if paar[0] == 1 and paar[1] == 3:
            pairs.append("13")
        if paar[0] == 1 and paar[1] == 4:
            pairs.append("14")

        if paar[0] == 2 and paar[1] == 0:
            pairs.append("20")
        if paar[0] == 2 and paar[1] == 1:
            pairs.append("21")
        if paar[0] == 2 and paar[1] == 2:
            pairs.append("22")
        if paar[0] == 2 and paar[1] == 3:
            pairs.append("23")
        if paar[0] == 2 and paar[1] == 4:
            pairs.append("24")

        if paar[0] == 3 and paar[1] == 0:
            pairs.append("30")
        if paar[0] == 3 and paar[1] == 1:
            pairs.append("31")
        if paar[0] == 3 and paar[1] == 2:
            pairs.append("32")
        if paar[0] == 3 and paar[1] == 3:
            pairs.append("33")
        if paar[0] == 3 and paar[1] == 4:
            pairs.append("34")

        if paar[0] == 4 and paar[1] == 0:
            pairs.append("40")
        if paar[0] == 4 and paar[1] == 1:
            pairs.append("41")
        if paar[0] == 4 and paar[1] == 2:
            pairs.append("42")
        if paar[0] == 4 and paar[1] == 3:
            pairs.append("43")
        if paar[0] == 4 and paar[1] == 4:
            pairs.append("44")
    sups_fertig = {str(x):klassen_files.count(x) for x in klassen_files}
    sups_two_fertig = {x:pairs.count(x) for x in pairs}
    # for i in range(5):
    #     print("Support(",klassen_categorien[i],") = ", sups_fertig[str(i)], "/",week_count , "=",  round(sups_fertig[str(i)]/week_count*100, 3), "%")
    # print("------------------------------------------------------------------------------------")
    # for i in range(5):
    #     for b in range(5):
    #         try:
    #             print("Support(", klassen_categorien[i], "->", klassen_categorien[b], ") = ", sups_two_fertig[str(i)+str(b)], "/", (week_count), "=", round(sups_two_fertig[str(i)+str(b)]/(week_count)*100, 2), "%")
    #         except:
    #             print("Support(", klassen_categorien[i], "->", klassen_categorien[b], ") = 0", "/",(week_count), "= 0%")
    # print("------------------------------------------------------------------------------------")
    # for i in range(5):
    #     for b in range(5):
    #         try:
    #             print("Conf(", klassen_categorien[i], "->", klassen_categorien[b], "= (", sups_two_fertig[str(i)+str(b)], "/", week_count, ") * (", week_count, "/", sups_fertig[str(i)], ") = ", round(sups_two_fertig[str(i)+str(b)]/sups_fertig[str(i)]*100, 2), "%")
    #         except:
    #             print("Conf(", klassen_categorien[i], "->", klassen_categorien[b], "= (", 0, "/", week_count, ") * (", week_count, "/", sups_fertig[str(i)], ") = 0%")

    print("----------------------LATEX--------------------------------------------------------------")
    for i in range(5):
        print("$sup(",klassen_categorien[i],") = \ frac{", sups_fertig[str(i)], "}{",week_count , "} \ approx",  round(sups_fertig[str(i)]/week_count*100, 3), "\% $ \ \ ")
    print("------------------------------------------------------------------------------------")
    for i in range(5):
        for b in range(5):
            try:
                print("$sup(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", sups_two_fertig[str(i)+str(b)], "}{", week_count, "} \ approx", round(sups_two_fertig[str(i)+str(b)]/(week_count)*100, 2), "\% $ \ \ ")
            except:
                print("$sup(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{0", "}{", week_count, "} = 0\% $ \ \ ")
    print("------------------------------------------------------------------------------------")
    for i in range(5):
        for b in range(5):
            try:
                print("$konf(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", sups_two_fertig[str(i)+str(b)], "}{", week_count, "} \ cdot  \ frac{", week_count, "}{", sups_fertig[str(i)], "} \ approx ", round(sups_two_fertig[str(i)+str(b)]/sups_fertig[str(i)]*100, 2), "\% $ \ \ ")
            except:
                print("$konf(", klassen_categorien[i], "\ text{\ textrightarrow\ }", klassen_categorien[b], ") = \ frac{", 0, "}{", week_count, "} \ cdot  \ frac{", week_count, "}{", sups_fertig[str(i)], "} = 0\% $ \ \ ")

# frage17()